In [ ]:
#!/usr/bin/env python3
import os
import cv2
import pandas as pd
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from collections import Counter, defaultdict
import yaml

# ---------------------------
# CONFIG (keep your paths/flow)
# ---------------------------
DATA_ROOT = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced"

IMG_DIRS = [
    f"{DATA_ROOT}/train/images",
    f"{DATA_ROOT}/valid/images",
    f"{DATA_ROOT}/test/images",
]

LBL_DIRS = [
    f"{DATA_ROOT}/train/labels",
    f"{DATA_ROOT}/valid/labels",
    f"{DATA_ROOT}/test/labels",
]

# ---- Output folder ----
OUT_FOLDER = f"{DATA_ROOT}/object_label_csv2"
os.makedirs(OUT_FOLDER, exist_ok=True)

OUT_CSV = f"{OUT_FOLDER}/object_and_labels.csv"

YOLO_DET_PATH = "yolo11m.pt"
DET_CONF_TH = 0.25

# ---------------------------
# LOAD Roboflow data.yaml and build index->numeric-level map
# ---------------------------
DATA_YAML = Path(f"{DATA_ROOT}/data.yaml")
idx_to_level = {}  # map label-index (int in txt) -> numeric flood level (int from 'Level X')

if DATA_YAML.exists():
    with open(DATA_YAML, "r") as fh:
        dy = yaml.safe_load(fh)
    rf_names = dy.get("names", [])
    # rf_names can be list like ['Level 0','Level 1','Level 10','Level 2', ...]
    for idx, name in enumerate(rf_names):
        # extract digits from the name string to get numeric flood level
        try:
            # pick continuous digits groups (handles 'Level 10' etc.)
            digits = ''.join(ch for ch in str(name) if ch.isdigit())
            num = int(digits) if digits != "" else idx
        except Exception:
            num = idx
        idx_to_level[int(idx)] = int(num)
else:
    # fallback: identity mapping (index -> index) so script still runs
    print(f"Warning: {DATA_YAML} not found. Using identity mapping index->level as fallback.")
    idx_to_level = {}  # leave empty, will fallback to lab_idx wherever used

# small sanity print (show mapping or note if empty)
if len(idx_to_level) > 0:
    print("Roboflow index -> flood_level mapping (sample):")
    for k in sorted(list(idx_to_level.keys()))[:20]:
        print(f"  index {k} -> flood_level {idx_to_level[k]}")
else:
    print("Roboflow mapping empty — script will use label-index as numeric flood level (lab_idx).")

# ---------------------------
# LOAD YOLO DETECTION MODEL
# ---------------------------
det_model = YOLO(YOLO_DET_PATH)


def detect_object_name(crop):
    """Return YOLO detected class name for the crop."""
    try:
        res = det_model.predict(crop, conf=DET_CONF_TH, imgsz=256, verbose=False)[0]
        if getattr(res, "boxes", None) is None or len(res.boxes) == 0:
            return "unknown"
        idx = res.boxes.conf.argmax()
        cls_id = int(res.boxes.cls[idx].item())
        cname = det_model.model.names[cls_id]
        return cname
    except Exception:
        return "unknown"


# ---------------------------
# QUICK PRE-SCAN: count labels in .txt files (expected counts)
# Note: now we map label index -> numeric flood level using idx_to_level
# ---------------------------
expected_counts = Counter()
total_label_lines = 0
label_files_scanned = 0

for lbl_dir in LBL_DIRS:
    if not os.path.isdir(lbl_dir):
        continue
    for fname in os.listdir(lbl_dir):
        if not fname.lower().endswith(".txt"):
            continue
        lbl_path = os.path.join(lbl_dir, fname)
        with open(lbl_path, "r") as f:
            lines = [ln.strip() for ln in f.read().splitlines() if ln.strip()]
        label_files_scanned += 1
        for ln in lines:
            parts = ln.split()
            if len(parts) < 1:
                continue
            try:
                lab_idx = int(parts[0])
                # map index -> numeric flood level via data.yaml mapping (safe fallback to lab_idx)
                flood_level = idx_to_level.get(lab_idx, lab_idx)
                expected_counts[flood_level] += 1
                total_label_lines += 1
            except Exception:
                # skip malformed label lines
                continue

print(f"Pre-scan: scanned {label_files_scanned} label files, total label lines = {total_label_lines}")
print("Expected label counts from .txt files (label-only, mapped via data.yaml):")
for lvl in sorted(expected_counts.keys()):
    print(f"  Level {lvl}: {expected_counts[lvl]}")

# ---------------------------
# MAIN EXTRACTION (preserve original flow)
# ---------------------------
rows = []
# We'll also keep a map of img -> number of label lines (for tracing)
img_label_line_counts = defaultdict(int)

for split_idx, (img_dir, lbl_dir) in enumerate(zip(IMG_DIRS, LBL_DIRS), start=1):
    # preserve original listing order (no forced sorting)
    images = [f for f in os.listdir(img_dir) if f.lower().endswith((".jpg", ".png"))]

    for img_index, img_name in enumerate(images):
        img_path = os.path.join(img_dir, img_name)
        lbl_path = os.path.join(lbl_dir, Path(img_name).stem + ".txt")

        if not os.path.exists(lbl_path):
            continue

        img = cv2.imread(img_path)
        if img is None:
            continue
        h, w = img.shape[:2]

        with open(lbl_path, "r") as f:
            lines = [ln.strip() for ln in f.read().splitlines() if ln.strip()]

        # track how many label lines existed for this image (for debugging)
        img_label_line_counts[img_path] = len(lines)

        for line_idx, ln in enumerate(lines):
            parts = ln.split()
            if len(parts) != 5:
                # keep behavior: only accept 5-part lines
                continue

            # parse flood level via Roboflow mapping:
            try:
                lab_idx = int(parts[0])
                flood_level = idx_to_level.get(lab_idx, lab_idx)  # safe fallback to lab_idx
            except Exception:
                # skip malformed flood level
                continue

            # bbox decode (same as your original)
            try:
                cx, cy, bw, bh = map(float, parts[1:])
            except Exception:
                continue

            cx *= w; cy *= h; bw *= w; bh *= h

            x1 = int(cx - bw/2)
            y1 = int(cy - bh/2)
            x2 = int(cx + bw/2)
            y2 = int(cy + bh/2)

            x1 = max(0, x1); y1 = max(0, y1)
            x2 = min(w-1, x2); y2 = min(h-1, y2)

            if x2 <= x1 or y2 <= y1:
                continue

            crop = img[y1:y2, x1:x2]

            object_name = detect_object_name(crop)

            # keep trace columns so we can later map any row back to exact file/line
            rows.append({
                "object_name": object_name.lower(),
                "flood_level": int(flood_level),
                "image_path": img_path,
                "split": ("train" if split_idx==1 else "valid" if split_idx==2 else "test"),
                "img_index_in_dir": img_index,
                "label_line_idx": line_idx,   # which line in the label file this came from
                "bbox_x1": x1, "bbox_y1": y1, "bbox_x2": x2, "bbox_y2": y2
            })

# ---------------------------
# SAVE CSV
# ---------------------------
df = pd.DataFrame(rows)
df.to_csv(OUT_CSV, index=False)

print("CSV SAVED:", OUT_CSV)
print("Total object rows written:", len(df))

# ---------------------------
# DIAGNOSTIC REPORT: compare expected (.txt) counts vs produced CSV counts
# ---------------------------
produced_counts = df["flood_level"].value_counts().to_dict()

print("\nProduced CSV flood level counts (from final CSV):")
for lvl in sorted(produced_counts.keys()):
    print(f"  Level {lvl}: {produced_counts[lvl]}")

# Compare (show diff = produced - expected)
print("\nDIFFERENCE (produced - expected):")
all_lvls = sorted(set(list(expected_counts.keys()) + list(produced_counts.keys())))
for lvl in all_lvls:
    exp = expected_counts.get(lvl, 0)
    prod = produced_counts.get(lvl, 0)
    diff = prod - exp
    note = ""
    if diff != 0:
        note = " <-- MISMATCH"
    print(f"  Level {lvl}: produced={prod} expected={exp} diff={diff}{note}")

# Extra trace: list any images where number of produced rows != number of label lines
img_rows_count = df.groupby("image_path").size().to_dict()
mismatched_images = []
for img_path, lbl_count in img_label_line_counts.items():
    prod_rows = img_rows_count.get(img_path, 0)
    if prod_rows != lbl_count:
        mismatched_images.append((img_path, lbl_count, prod_rows))

if mismatched_images:
    print("\nImages where produced rows != label lines (image_path, label_lines, produced_rows):")
    for t in mismatched_images[:50]:
        print(" ", t)
else:
    print("\nAll images produced same number of object rows as label lines (good).")

In [ ]:
import pandas as pd

CSV_PATH = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced/object_label_csv2/object_and_labels.csv"

df = pd.read_csv(CSV_PATH)

# group by object_name and flood_level
grouped = df.groupby(["object_name", "flood_level"]).size().reset_index(name="count")

# print nicely
for obj in grouped["object_name"].unique():
    print(f"\n=== OBJECT: {obj} ===")
    sub = grouped[grouped["object_name"] == obj].sort_values("flood_level")
    for _, row in sub.iterrows():
        level = row["flood_level"]
        cnt = row["count"]
        print(f"  Level {level}: {cnt} samples")

In [ ]:
import os
import pandas as pd

DATA_ROOT = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced"

CSV_PATH = f"{DATA_ROOT}/object_label_csv2/object_and_labels.csv"

IMG_DIRS = [
    f"{DATA_ROOT}/train/images",
    f"{DATA_ROOT}/valid/images",
    f"{DATA_ROOT}/test/images",
]

# -----------------------------
# 1. Count unique image paths in CSV
# -----------------------------
df = pd.read_csv(CSV_PATH)
unique_csv_images = df["image_path"].unique()
num_unique_csv_images = len(unique_csv_images)

print("Unique images in CSV:", num_unique_csv_images)

# -----------------------------
# 2. Count actual images in dataset folders
# -----------------------------
actual_images = []

for d in IMG_DIRS:
    for f in os.listdir(d):
        if f.lower().endswith((".jpg", ".jpeg", ".png")):
            actual_images.append(os.path.join(d, f))

actual_images = sorted(actual_images)
num_actual_images = len(actual_images)

print("Actual images in dataset:", num_actual_images)

# -----------------------------
# 3. Compare
# -----------------------------
if num_unique_csv_images == num_actual_images:
    print("\n✅ MATCH: CSV contains all images.")
else:
    print("\n❌ MISMATCH:")
    print("Images missing in CSV:", num_actual_images - num_unique_csv_images)

In [ ]:
import pandas as pd

CSV_PATH = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced/object_label_csv2/object_and_labels_filtered.csv"

df = pd.read_csv(CSV_PATH)

unique_classes = df["object_name"].unique()
unique_levels = df["flood_level"].unique()

print("UNIQUE OBJECT CLASSES:")
for c in unique_classes:
    print(" -", c)

print("\nFLOOD LEVEL COUNTS:")
level_counts = df["flood_level"].value_counts().sort_index()

for level, count in level_counts.items():
    print(f"Level {level}: {count} samples")

print("\nTOTAL UNIQUE FLOOD LEVELS:", len(unique_levels))
print("TOTAL SAMPLES:", len(df))

In [ ]:
import pandas as pd

CSV_IN = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced/object_label_csv2/object_and_labels.csv"
CSV_OUT = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced/object_label_csv2/object_and_labels_filtered.csv"

# classes to keep
KEEP_CLASSES = ["person", "motorcycle", "car", "truck", "bus", "bicycle"]

df = pd.read_csv(CSV_IN)

# keep rows with allowed object names
df_filtered = df[df["object_name"].isin(KEEP_CLASSES)].copy()

df_filtered.to_csv(CSV_OUT, index=False)

print("Filtered CSV saved to:", CSV_OUT)
print("Rows kept:", len(df_filtered))

In [ ]:
#!/usr/bin/env python3
"""
Script Final — Compute physics + semantics per GT from object_and_labels_filtered.csv
 - Uses filtered CSV (no re-detection)
 - Per-image: water segmentation + pose (if weights present)
 - Per-GT: compute base features + semantic features for 6 classes:
    person, car, truck, bus, motorcycle, bicycle
 - Saves CSV with original columns plus features
"""

import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from ultralytics import YOLO
import yaml

# -----------------------
# CONFIG (edit if needed)
# -----------------------
DATA_ROOT = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced"
IN_FILTERED_CSV = f"{DATA_ROOT}/object_label_csv2/object_and_labels_filtered.csv"
OUT_FOLDER = f"{DATA_ROOT}/object_label_csv2"
os.makedirs(OUT_FOLDER, exist_ok=True)
OUT_CSV = f"{OUT_FOLDER}/object_and_labels_final_44feat.csv"

IMG_DIRS = {
    "train": f"{DATA_ROOT}/train/images",
    "valid": f"{DATA_ROOT}/valid/images",
    "test":  f"{DATA_ROOT}/test/images",
}

LBL_DIRS = {
    "train": f"{DATA_ROOT}/train/labels",
    "valid": f"{DATA_ROOT}/valid/labels",
    "test":  f"{DATA_ROOT}/test/labels",
}

# models (change paths if needed)
YOLO_POSE_PATH = "yolo11m-pose.pt"   # optional
YOLO_WSEG_PATH = "/home/arnab/Desktop/yolo/data/Flood_model/UrbanFlood_WaterSeg2/yolo11m-water-seg2/weights/best.pt"  # optional

# thresholds
POSE_CONF_TH = 0.25
WATER_CONF_TH = 0.25

# -----------------------
# data.yaml mapping: index -> numeric flood level
# -----------------------
DATA_YAML = Path(f"{DATA_ROOT}/data.yaml")
idx_to_level = {}
if DATA_YAML.exists():
    try:
        with open(DATA_YAML, "r") as fh:
            dy = yaml.safe_load(fh)
        rf_names = dy.get("names", [])
        for idx, name in enumerate(rf_names):
            try:
                digits = ''.join(ch for ch in str(name) if ch.isdigit())
                num = int(digits) if digits != "" else idx
            except Exception:
                num = idx
            idx_to_level[int(idx)] = int(num)
    except Exception:
        idx_to_level = {}
else:
    idx_to_level = {}

if len(idx_to_level) > 0:
    print("Loaded data.yaml mapping (index -> flood level) sample:")
    for k in sorted(list(idx_to_level.keys()))[:20]:
        print(f"  {k} -> {idx_to_level[k]}")
else:
    print("No data.yaml mapping found — will use label index as numeric flood level (lab_idx).")

# height priors (cm) — UPDATED per your request
CLASS_HEIGHT_PRIORS_CM = {
    "person":170, "car":150, "bus":300, "truck":300,
    "bicycle":100, "motorcycle":110,
}
DEFAULT_CLASS_HEIGHT = 150

# Allowed classes (we keep only these six in semantics)
ALLOWED_CLASSES = ["person","car","truck","bus","motorcycle","bicycle"]

# -----------------------
# FEATURE LIST (includes geometry, water, physics, and per-class semantic groups)
# NOTE: human_* renamed to person_* as requested
# -----------------------
ALL_FEATURES = [
    # geometry
    "box_x1","box_y1","box_x2","box_y2","box_w","box_h","box_area",
    # water/base
    "water_pixels","water_area_frac","water_top_y","water_height_frac",
    "sgf_mean","sgf_max","WOMI","submergence_ratio","bbox_ratio",
    "water_frac_ratio","waterline_norm",
    # physics
    "estimated_depth_cm","ref_height_cm","physics_residual",
    # person semantics
    "person_sub_ankle","person_sub_knee","person_sub_hip","person_sub_chest","person_sub_head","person_depth_norm",
    # NEW person fine-grained semantics
    "person_sub_mid_thigh","person_sub_upper_thigh","person_sub_lower_waist","person_depth_fine_ratio",
    # generic submergence flags
    "obj_sub_20pct","obj_sub_50pct","obj_sub_80pct",
    # car semantics
    "car_sub_wheel","car_sub_mid","car_sub_window","car_sub_roof","car_depth_norm","car_sub_half_door",
    # bus semantics
    "bus_sub_wheel","bus_sub_mid","bus_sub_window","bus_sub_roof","bus_depth_norm",
    # truck semantics
    "truck_sub_wheel","truck_sub_mid","truck_sub_window","truck_sub_roof","truck_depth_norm","truck_sub_hubcap",
    # motorcycle semantics
    "motorcycle_sub_wheel","motorcycle_sub_seat","motorcycle_sub_handle","motorcycle_depth_norm","motorcycle_sub_engine_level",
    # bicycle semantics
    "bicycle_sub_wheel","bicycle_sub_seat","bicycle_sub_handle","bicycle_depth_norm"
]

# -----------------------
# UTILITIES
# -----------------------
def safe_name(s):
    if s is None:
        return "unknown"
    return str(s).strip().lower()

def nanmean_safe(values, fallback):
    arr = []
    for v in values:
        if v is None:
            arr.append(np.nan)
        else:
            try:
                arr.append(float(v))
            except:
                arr.append(np.nan)
    out = np.nanmean(arr)
    if np.isnan(out):
        return fallback
    return out

def compute_iou(b1, b2):
    x1 = max(b1[0], b2[0]); y1 = max(b1[1], b2[1])
    x2 = min(b1[2], b2[2]); y2 = min(b1[3], b2[3])
    inter = max(0, x2-x1) * max(0, y2-y1)
    if inter <= 0: return 0.0
    a1 = max(1e-9, (b1[2]-b1[0])*(b1[3]-b1[1])); a2 = max(1e-9, (b2[2]-b2[0])*(b2[3]-b2[1]))
    return inter / (a1 + a2 - inter + 1e-9)

# -----------------------
# Models: load pose & wseg (optional)
# -----------------------
def load_models():
    pose = YOLO(YOLO_POSE_PATH) if os.path.exists(YOLO_POSE_PATH) else None
    wseg = YOLO(YOLO_WSEG_PATH) if os.path.exists(YOLO_WSEG_PATH) else None
    return pose, wseg

def run_water_seg(wseg_model, image):
    try:
        if wseg_model is None:
            return np.zeros(image.shape[:2], dtype=np.uint8)
        res = wseg_model.predict(image, imgsz=1024, conf=WATER_CONF_TH, verbose=False)[0]
        if getattr(res, "masks", None) is None:
            return np.zeros(image.shape[:2], dtype=np.uint8)
        mask = res.masks.data.cpu().numpy()
        if mask.ndim == 3:
            mask = (mask > 0.5).any(axis=0)
        else:
            mask = (mask > 0.5)
        return mask.astype(np.uint8)
    except Exception:
        return np.zeros(image.shape[:2], dtype=np.uint8)

def extract_poses(pose_model, image):
    if pose_model is None:
        return []
    try:
        res = pose_model.predict(image, imgsz=640, conf=POSE_CONF_TH, verbose=False)[0]
        if getattr(res, "keypoints", None) is None:
            return []
        return res.keypoints.data.cpu().numpy()
    except Exception:
        return []

def nearest_pose(box, poses):
    if len(poses) == 0:
        return None
    x1, y1, x2, y2 = box
    cx = (x1 + x2) / 2; cy = (y1 + y2) / 2
    best_pose = None; best_d = 1e12
    for p in poses:
        try:
            xs = p[:,0]; ys = p[:,1]
            xs = xs[~np.isnan(xs)]; ys = ys[~np.isnan(ys)]
            if len(xs) == 0:
                px, py = cx, cy
            else:
                px, py = xs.mean(), ys.mean()
        except:
            px, py = cx, cy
        d = (px - cx)**2 + (py - cy)**2
        if d < best_d:
            best_d = d; best_pose = p
    return best_pose

# -----------------------
# Compute base physics features for single box
# -----------------------
def compute_base_features(box, mask, img_w, img_h):
    x1, y1, x2, y2 = map(int, box)
    bw = max(1, x2 - x1); bh = max(1, y2 - y1)
    box_area = bw * bh
    if (y2 > y1) and (x2 > x1):
        submask = mask[y1:y2, x1:x2]
    else:
        submask = np.zeros((0,0), dtype=np.uint8)
    water_pixels = int(submask.sum())
    water_area_frac = water_pixels / (box_area + 1e-9)
    ys = np.where(submask > 0)[0]
    if len(ys) > 0:
        top_local = ys.min() + y1
        water_height_frac = (y2 - top_local) / (bh + 1e-9)
    else:
        top_local = y2
        water_height_frac = 0.0
    if submask.size > 0:
        sm = (submask * 255).astype(np.uint8)
        gx = cv2.Sobel(sm, cv2.CV_32F, 1, 0, ksize=3)
        gy = cv2.Sobel(sm, cv2.CV_32F, 0, 1, ksize=3)
        grad = np.sqrt(gx*gx + gy*gy)
        sgf_mean = float(grad.mean()); sgf_max = float(grad.max())
    else:
        sgf_mean, sgf_max = 0.0, 0.0
    water_top_y = int(top_local)
    return {
        "box_x1": x1, "box_y1": y1, "box_x2": x2, "box_y2": y2,
        "box_w": bw, "box_h": bh, "box_area": box_area,
        "water_pixels": water_pixels, "water_area_frac": water_area_frac,
        "water_top_y": water_top_y, "water_height_frac": water_height_frac,
        "sgf_mean": sgf_mean, "sgf_max": sgf_max, "WOMI": water_area_frac,
        "submergence_ratio": water_height_frac, "bbox_ratio": bh / (bw + 1e-9),
        "water_frac_ratio": water_area_frac, "waterline_norm": water_top_y / (img_h + 1e-9)
    }

# -----------------------
# Semantics and vehicle/person blocks (unchanged logic, renamed human->person)
# -----------------------
def person_semantics(pose, water_top_y, box):
    x1, y1, x2, y2 = map(int, box)
    bh = max(1, y2 - y1)
    ankle_b = y1 + 0.95 * bh; knee_b  = y1 + 0.75 * bh
    hip_b   = y1 + 0.55 * bh; chest_b = y1 + 0.35 * bh; head_b  = y1 + 0.15 * bh
    mid_thigh_b   = y1 + 0.80 * bh
    upper_thigh_b = y1 + 0.70 * bh
    lower_waist_b = y1 + 0.60 * bh
    ankle, knee, hip, chest, head = ankle_b, knee_b, hip_b, chest_b, head_b
    if pose is not None:
        try:
            conf = pose[:,2]
        except:
            conf = np.zeros(pose.shape[0])
        def gety(idx):
            try:
                if conf[idx] > POSE_CONF_TH:
                    return float(pose[idx,1])
            except:
                return None
        ankle = nanmean_safe([gety(15), gety(16)], ankle_b)
        knee  = nanmean_safe([gety(13), gety(14)], knee_b)
        hip   = nanmean_safe([gety(11), gety(12)], hip_b)
        chest = nanmean_safe([gety(5),  gety(6)],  chest_b)
        head  = nanmean_safe([gety(0)],            head_b)
    flags = {
        "person_sub_ankle": int(water_top_y > ankle),
        "person_sub_knee":  int(water_top_y > knee),
        "person_sub_hip":   int(water_top_y > hip),
        "person_sub_chest": int(water_top_y > chest),
        "person_sub_head":  int(water_top_y > head),
    }
    flags.update({
        "person_sub_mid_thigh":   int(water_top_y > mid_thigh_b),
        "person_sub_upper_thigh": int(water_top_y > upper_thigh_b),
        "person_sub_lower_waist": int(water_top_y > lower_waist_b),
    })
    denom = head - ankle
    if denom <= 0:
        fine_ratio = 0.0
    else:
        fine_ratio = (water_top_y - lower_waist_b) / denom
        fine_ratio = float(np.clip(fine_ratio, 0.0, 1.0))
    flags["person_depth_fine_ratio"] = fine_ratio
    if denom <= 0:
        depth = 0.0
    else:
        depth = (water_top_y - ankle) / denom
        depth = float(np.clip(depth, 0.0, 1.0))
    flags["person_depth_norm"] = depth
    return flags

def car_semantics(box, water_top_y):
    x1,y1,x2,y2 = map(int, box)
    bh = max(1, y2 - y1)
    wheel = y1 + int(0.85*bh); mid = y1 + int(0.50*bh)
    window = y1 + int(0.65*bh); roof = y1
    half_door = y1 + int(0.45 * bh)
    flags = {
        "car_sub_wheel": int(water_top_y > wheel),
        "car_sub_mid": int(water_top_y > mid),
        "car_sub_window": int(water_top_y > window),
        "car_sub_roof": int(water_top_y > roof),
        "car_sub_half_door": int(water_top_y > half_door),
    }
    denom = roof - wheel
    if denom <= 0:
        flags["car_depth_norm"] = 0.0
    else:
        flags["car_depth_norm"] = float(np.clip((water_top_y - wheel)/denom, 0.0, 1.0))
    return flags

def bus_semantics(box, water_top_y):
    x1,y1,x2,y2 = map(int, box)
    bh = max(1, y2 - y1)
    wheel = y1 + int(0.85*bh); mid = y1 + int(0.50*bh)
    window = y1 + int(0.65*bh); roof = y1
    flags = {
        "bus_sub_wheel": int(water_top_y > wheel),
        "bus_sub_mid": int(water_top_y > mid),
        "bus_sub_window": int(water_top_y > window),
        "bus_sub_roof": int(water_top_y > roof)
    }
    denom = roof - wheel
    if denom <= 0:
        flags["bus_depth_norm"] = 0.0
    else:
        flags["bus_depth_norm"] = float(np.clip((water_top_y - wheel)/denom, 0.0, 1.0))
    return flags

def truck_semantics(box, water_top_y):
    x1,y1,x2,y2 = map(int, box)
    bh = max(1, y2 - y1)
    wheel = y1 + int(0.85*bh); mid = y1 + int(0.50*bh)
    window = y1 + int(0.65*bh); roof = y1
    hubcap = y1 + int(0.90 * bh)
    flags = {
        "truck_sub_wheel": int(water_top_y > wheel),
        "truck_sub_mid": int(water_top_y > mid),
        "truck_sub_window": int(water_top_y > window),
        "truck_sub_roof": int(water_top_y > roof),
        "truck_sub_hubcap": int(water_top_y > hubcap),
    }
    denom = roof - wheel
    if denom <= 0:
        flags["truck_depth_norm"] = 0.0
    else:
        flags["truck_depth_norm"] = float(np.clip((water_top_y - wheel)/denom, 0.0, 1.0))
    return flags

def motorcycle_semantics(box, water_top_y):
    x1,y1,x2,y2 = map(int, box)
    bh = max(1, y2 - y1)
    wheel = y1 + int(0.90*bh); seat = y1 + int(0.60*bh); handle = y1 + int(0.30*bh)
    engine = y1 + int(0.50 * bh)
    flags = {
        "motorcycle_sub_wheel": int(water_top_y > wheel),
        "motorcycle_sub_seat": int(water_top_y > seat),
        "motorcycle_sub_handle": int(water_top_y > handle),
        "motorcycle_sub_engine_level": int(water_top_y > engine),
    }
    denom = handle - wheel
    if denom <= 0:
        flags["motorcycle_depth_norm"] = 0.0
    else:
        flags["motorcycle_depth_norm"] = float(np.clip((water_top_y - wheel)/denom, 0.0, 1.0))
    return flags

def bicycle_semantics(box, water_top_y):
    x1,y1,x2,y2 = map(int, box)
    bh = max(1, y2 - y1)
    wheel = y1 + int(0.90*bh); seat = y1 + int(0.60*bh); handle = y1 + int(0.30*bh)
    flags = {
        "bicycle_sub_wheel": int(water_top_y > wheel),
        "bicycle_sub_seat": int(water_top_y > seat),
        "bicycle_sub_handle": int(water_top_y > handle)
    }
    denom = handle - wheel
    if denom <= 0:
        flags["bicycle_depth_norm"] = 0.0
    else:
        flags["bicycle_depth_norm"] = float(np.clip((water_top_y - wheel)/denom, 0.0, 1.0))
    return flags

def generic_semantics(subratio):
    return {
        "obj_sub_20pct": int(subratio > 0.20),
        "obj_sub_50pct": int(subratio > 0.50),
        "obj_sub_80pct": int(subratio > 0.80),
    }

def estimate_depth_cm(subratio, cname):
    cname = safe_name(cname)
    ref = CLASS_HEIGHT_PRIORS_CM.get(cname, DEFAULT_CLASS_HEIGHT)
    return float(ref * subratio), float(ref)

# -----------------------
# Assemble single GT row
# -----------------------
def process_gt_box(rgb, box, gt_level, cname_from_csv, poses_arr, mask, img_w, img_h, obj_idx):
    x1,y1,x2,y2 = map(int, box)
    if x2 <= x1 or y2 <= y1:
        return None
    base = compute_base_features([x1,y1,x2,y2], mask, img_w, img_h)
    pose_for_obj = nearest_pose([x1,y1,x2,y2], poses_arr) if len(poses_arr) > 0 else None
    cname = safe_name(cname_from_csv)
    sem = {}
    if cname == "person":
        sem.update(person_semantics(pose_for_obj, base["water_top_y"], [x1,y1,x2,y2]))
    elif cname == "car":
        sem.update(car_semantics([x1,y1,x2,y2], base["water_top_y"]))
    elif cname == "bus":
        sem.update(bus_semantics([x1,y1,x2,y2], base["water_top_y"]))
    elif cname == "truck":
        sem.update(truck_semantics([x1,y1,x2,y2], base["water_top_y"]))
    elif cname == "motorcycle":
        sem.update(motorcycle_semantics([x1,y1,x2,y2], base["water_top_y"]))
    elif cname == "bicycle":
        sem.update(bicycle_semantics([x1,y1,x2,y2], base["water_top_y"]))
    sem.update(generic_semantics(base["submergence_ratio"]))
    est_cm, ref_cm = estimate_depth_cm(base["submergence_ratio"], cname)
    row = {
        "image_path": None,
        "object_name": cname,
        "flood_level": int(gt_level),
        "obj_id": int(obj_idx)
    }
    for k,v in base.items():
        row[k] = float(v) if isinstance(v, (int, float, np.floating, np.integer)) else v
    row["estimated_depth_cm"] = float(est_cm)
    row["ref_height_cm"] = float(ref_cm)
    row["physics_residual"] = 0.0
    for f in ALL_FEATURES:
        if f not in row:
            row[f] = 0.0
    for k,v in sem.items():
        row[k] = float(v)
    for k in ALL_FEATURES:
        if isinstance(row.get(k, None), (int, np.integer)):
            row[k] = float(row[k])
    return row

# -----------------------
# Cleaning utility for final df
# -----------------------
def clean_df(df):
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.fillna(0)
    return df

# -----------------------
# MAIN: read filtered CSV and compute features
# -----------------------
def main():
    if not os.path.exists(IN_FILTERED_CSV):
        print("Filtered CSV not found:", IN_FILTERED_CSV)
        return

    df_filtered = pd.read_csv(IN_FILTERED_CSV)
    print("Filtered rows:", len(df_filtered))

    # build assignment map: (image_path, flood_level) -> list of object_name in original order
    assign_map = {}
    for _, r in df_filtered.iterrows():
        ip = r["image_path"]
        # safer parsing if flood_level is string/float
        try:
            fl = int(float(r["flood_level"]))
        except Exception:
            fl = int(r["flood_level"])
        key = (ip, fl)
        assign_map.setdefault(key, []).append(r["object_name"])

    need_counts = {k: len(v) for k,v in assign_map.items()}
    print("Unique (image,level) groups:", len(need_counts))

    # load models
    print("Loading pose & waterseg models (if available)...")
    pose_model, wseg_model = load_models()
    if pose_model is None:
        print("NOTE: Pose model not found; pose features will be skipped.")
    if wseg_model is None:
        print("NOTE: Water segmentation model not found; water masks will be zeros.")

    out_rows = []
    processed_counts = {}

    unique_images = sorted(df_filtered["image_path"].unique())
    print("Images to process:", len(unique_images))

    for img_path in tqdm(unique_images):
        img_name = Path(img_path).name
        # find label file
        lbl_path = None
        for d in LBL_DIRS.values():
            candidate = os.path.join(d, Path(img_name).stem + ".txt")
            if os.path.exists(candidate):
                lbl_path = candidate
                break
        if lbl_path is None:
            # no label file found — skip
            print("Label missing for:", img_path)
            continue

        img = cv2.imread(img_path)
        if img is None:
            print("Failed to load image:", img_path)
            continue
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h_img, w_img = rgb.shape[:2]

        # read GT boxes in label file
        with open(lbl_path, "r") as f:
            lines = [ln.strip() for ln in f.read().splitlines() if ln.strip()]

        gt_boxes = []
        for ln in lines:
            parts = ln.split()
            if len(parts) != 5:
                continue
            # map label-index via data.yaml mapping if available
            try:
                lab_idx = int(parts[0])
                cls_id = int(idx_to_level.get(lab_idx, lab_idx))
            except Exception:
                # fallback: try direct int cast (original behaviour)
                try:
                    cls_id = int(parts[0])
                except:
                    continue
            cx = float(parts[1]) * w_img; cy = float(parts[2]) * h_img
            bw = float(parts[3]) * w_img; bh = float(parts[4]) * h_img
            x1 = int(cx - bw/2); y1 = int(cy - bh/2); x2 = int(cx + bw/2); y2 = int(cy + bh/2)
            x1 = max(0, x1); y1 = max(0, y1); x2 = min(w_img-1, x2); y2 = min(h_img-1, y2)
            if x2 <= x1 or y2 <= y1:
                continue
            gt_boxes.append((x1,y1,x2,y2,cls_id))

        if len(gt_boxes) == 0:
            continue

        # compute image-level masks and poses once
        mask = run_water_seg(wseg_model, rgb) if wseg_model is not None else np.zeros((h_img, w_img), dtype=np.uint8)
        poses = []
        if pose_model is not None:
            try:
                pres = pose_model.predict(rgb, imgsz=640, conf=POSE_CONF_TH, verbose=False)[0]
                if getattr(pres, "keypoints", None) is not None:
                    arr = pres.keypoints.data.cpu().numpy()
                    for i in range(arr.shape[0]):
                        poses.append(arr[i])
            except Exception:
                poses = []

        # iterate GT boxes in file order, and assign only if requested by assign_map
        for idx, (x1,y1,x2,y2,cls_id) in enumerate(gt_boxes):
            key = (img_path, int(cls_id))
            needed = need_counts.get(key, 0)
            already = processed_counts.get(key, 0)
            if already >= needed:
                continue
            name_list = assign_map.get(key, [])
            if len(name_list) == 0:
                # nothing to assign
                continue
            cname = name_list.pop(0)
            cname_norm = safe_name(cname)
            row = process_gt_box(rgb, (x1,y1,x2,y2), cls_id, cname_norm, poses, mask, w_img, h_img, already)
            if row is None:
                continue
            row["image_path"] = img_path
            row["object_name"] = cname_norm
            out_rows.append(row)
            processed_counts[key] = processed_counts.get(key, 0) + 1

    print("Feature rows produced:", len(out_rows))

    # check unmet groups
    unmet = []
    for k, need in need_counts.items():
        got = processed_counts.get(k, 0)
        if got < need:
            unmet.append((k, need, got))
    if len(unmet) > 0:
        print("WARNING: some (image,level) groups had fewer GT boxes than rows requested in filtered CSV:")
        for (ip, lvl), need, got in unmet:
            print(f"  {ip} | level={lvl} -> needed {need}, got {got}")

    if len(out_rows) == 0:
        print("No feature rows produced. Exiting.")
        return

    df_out = pd.DataFrame(out_rows)
    df_out = clean_df(df_out)

    # ensure columns order: image_path, object_name, flood_level, ALL_FEATURES...
    cols = ["image_path", "object_name", "flood_level"] + ALL_FEATURES
    # keep only columns present in df_out (defensive)
    cols = [c for c in cols if c in df_out.columns]
    df_out = df_out[cols]

    df_out.to_csv(OUT_CSV, index=False)
    print("Saved final CSV:", OUT_CSV)
    print("Done.")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd

csv_path = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced/object_label_csv2/object_and_labels_final_44feat.csv"

df = pd.read_csv(csv_path)

print("\n===== UNIQUE CLASSES =====")
print(df["object_name"].unique())

print("\n===== UNIQUE FLOOD LEVELS =====")
print(sorted(df["flood_level"].unique()))

print("\n===== CSV HEAD (FIRST 5 ROWS) =====")
print(df.head())

print("\n===== COLUMN NAMES =====")
print(df.columns.tolist())

In [ ]:
import pandas as pd

csv_path = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced/object_label_csv2/object_and_labels_final_44feat.csv"

df = pd.read_csv(csv_path)

print("Flood level counts (0–10):\n")
counts = df["flood_level"].value_counts().sort_index()

for level, count in counts.items():
    print(f"{level} = {count}")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

CSV_IN = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced/object_label_csv2/object_and_labels_final_44feat.csv"

OUT_TRAIN = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced/object_label_csv2/train_balanced_80.csv"
OUT_TEST  = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced/object_label_csv2/test_balanced_20.csv"

df = pd.read_csv(CSV_IN)

print("Original counts:")
print(df["flood_level"].value_counts().sort_index())

# ------------------------------------------------------
# 1. DETERMINE MAX COUNT (for oversampling target)
# ------------------------------------------------------
level_counts = df["flood_level"].value_counts()
max_count = level_counts.max()

print("\nMax samples among levels =", max_count)

# ------------------------------------------------------
# 2. OVERSAMPLE EACH LEVEL TO max_count
# ------------------------------------------------------
balanced_parts = []

for level in sorted(df["flood_level"].unique()):
    sub = df[df["flood_level"] == level]
    if len(sub) < max_count:
        # oversample with replacement
        sub_over = sub.sample(max_count, replace=True, random_state=42)
    else:
        sub_over = sub
    balanced_parts.append(sub_over)

df_balanced = pd.concat(balanced_parts, ignore_index=True)

print("\nAfter oversampling:")
print(df_balanced["flood_level"].value_counts().sort_index())

# ------------------------------------------------------
# 3. STRATIFIED TRAIN/TEST SPLIT
# ------------------------------------------------------
train_df, test_df = train_test_split(
    df_balanced,
    test_size=0.10,
    random_state=42,
    shuffle=True,
    stratify=df_balanced["flood_level"]
)

# ------------------------------------------------------
# 4. SAVE OUTPUT CSVs
# ------------------------------------------------------
train_df.to_csv(OUT_TRAIN, index=False)
test_df.to_csv(OUT_TEST, index=False)

print("\nSaved train:", OUT_TRAIN, "rows:", len(train_df))
print("Saved test:", OUT_TEST, "rows:", len(test_df))

print("\nTRAIN distribution:")
print(train_df["flood_level"].value_counts().sort_index())

print("\nTEST distribution:")
print(test_df["flood_level"].value_counts().sort_index())

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import os

# -----------------------------
# CONFIG
# -----------------------------
TRAIN_IN = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced/object_label_csv2/train_balanced_80.csv"
OUT_DIR = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced/object_label_csv2/kfolds_train_only"
os.makedirs(OUT_DIR, exist_ok=True)

K = 5  # change if needed (e.g., 5, 10, etc.)

# -----------------------------
# LOAD TRAIN CSV
# -----------------------------
df = pd.read_csv(TRAIN_IN)

print("Balanced TRAIN level counts:")
print(df["flood_level"].value_counts().sort_index())

# --------------------------------------------------------
# 1. STRATIFIED K-FOLD SPLIT ON TRAIN ONLY
# --------------------------------------------------------
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

X = df
y = df["flood_level"]

fold = 1
for train_idx, val_idx in skf.split(X, y):

    df_train_k = df.iloc[train_idx].reset_index(drop=True)
    df_val_k   = df.iloc[val_idx].reset_index(drop=True)

    train_path = f"{OUT_DIR}/kfold_train_{fold}.csv"
    val_path   = f"{OUT_DIR}/kfold_val_{fold}.csv"

    df_train_k.to_csv(train_path, index=False)
    df_val_k.to_csv(val_path, index=False)

    print(f"\n===== FOLD {fold} =====")
    print("Train rows:", len(df_train_k))
    print(df_train_k["flood_level"].value_counts().sort_index())

    print("\nVal rows:", len(df_val_k))
    print(df_val_k["flood_level"].value_counts().sort_index())

    fold += 1

print("\nK-fold CSVs saved in:", OUT_DIR)

In [ ]:
# Cell 1 - setup
import os, sys, math, random, time
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as T
from torchvision.models import resnet50, ResNet50_Weights

import cv2
from PIL import Image

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import joblib

# NGBoost (replaces XGB)
from ngboost import NGBClassifier
from ngboost.distns import k_categorical
from ngboost.scores import LogScore

# config
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Paths (update for new 50+ features)
DATA_ROOT = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced/object_label_csv2ConvNeXT_NGBoost_FShL+llm"
K_FOLD_DIR = f"{DATA_ROOT}/kfolds_train_only"
TEST_CSV = f"{DATA_ROOT}/test_balanced_20.csv"

# new output folder
OUT_DIR = f"{DATA_ROOT}/hybrid_resnet50_xgb_fusion_v1"
os.makedirs(OUT_DIR, exist_ok=True)

# YOLO weights for inference (feature computation)
YOLO_DET_PATH  = "yolo11m.pt"
YOLO_POSE_PATH = "yolo11m-pose.pt"
YOLO_WSEG_PATH = "/home/arnab/Desktop/yolo/data/Flood_model/UrbanFlood_WaterSeg/yolo11m-water-seg/weights/best.pt"

# model/hyperparams
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 30
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-4
SCHEDULER = "cosine"   # "cosine","onecycle","step"
NUM_CLASSES = 11

ACT = nn.SiLU  # option A

In [ ]:
# Cell 2 - feature list + utils

ALL_FEATURES = [
    # geometry
    "box_x1","box_y1","box_x2","box_y2","box_w","box_h","box_area",

    # water/base
    "water_pixels","water_area_frac","water_top_y","water_height_frac",
    "sgf_mean","sgf_max","WOMI","submergence_ratio","bbox_ratio",
    "water_frac_ratio","waterline_norm",

    # physics
    "estimated_depth_cm","ref_height_cm","physics_residual",

    # person semantics (renamed from human_*)
    "person_sub_ankle","person_sub_knee","person_sub_hip",
    "person_sub_chest","person_sub_head","person_depth_norm",

    # 🔥 NEW PERSON SEMANTICS
    "person_sub_mid_thigh",
    "person_sub_upper_thigh",
    "person_sub_lower_waist",
    "person_depth_fine_ratio",

    # generic submergence flags
    "obj_sub_20pct","obj_sub_50pct","obj_sub_80pct",

    # car semantics
    "car_sub_wheel","car_sub_mid","car_sub_window","car_sub_roof","car_depth_norm",

    # 🔥 NEW CAR SEMANTIC
    "car_sub_half_door",

    # bus semantics
    "bus_sub_wheel","bus_sub_mid","bus_sub_window","bus_sub_roof","bus_depth_norm",

    # truck semantics
    "truck_sub_wheel","truck_sub_mid","truck_sub_window","truck_sub_roof","truck_depth_norm",

    # 🔥 NEW TRUCK SEMANTIC
    "truck_sub_hubcap",

    # motorcycle semantics
    "motorcycle_sub_wheel","motorcycle_sub_seat","motorcycle_sub_handle","motorcycle_depth_norm",

    # 🔥 NEW MOTORCYCLE SEMANTIC
    "motorcycle_sub_engine_level",

    # bicycle semantics
    "bicycle_sub_wheel","bicycle_sub_seat","bicycle_sub_handle","bicycle_depth_norm"
]

def safe_load_image_rgb(path):
    img = cv2.imread(path)
    if img is None:
        try:
            pil = Image.open(path).convert("RGB")
            arr = np.array(pil)[:, :, ::-1]
            return arr
        except:
            return np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.uint8)
    return img[:, :, ::-1]

# image transforms
train_transform = T.Compose([
    T.ToPILImage(),
    T.RandomResizedCrop(IMAGE_SIZE, scale=(0.8,1.0)),
    T.RandomHorizontalFlip(),
    T.ColorJitter(0.2,0.2,0.2,0.02),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

val_transform = T.Compose([
    T.ToPILImage(),
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

In [ ]:
# Cell 3 - dataset
class HybridCropDataset(Dataset):
    def __init__(self, csv_path, feature_cols, transform=None, scaler: StandardScaler=None):
        self.df = pd.read_csv(csv_path)

        # Ensure ALL_FEATURES exist (including new ones)
        for c in feature_cols:
            if c not in self.df.columns:
                self.df[c] = 0.0

        self.feature_cols = feature_cols
        self.transform = transform
        self.scaler = scaler

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # -------- load image --------
        img_path = row["image_path"]
        img = safe_load_image_rgb(img_path)
        h, w = img.shape[:2]

        # -------- crop by GT box --------
        x1 = int(row["box_x1"]); y1 = int(row["box_y1"])
        x2 = int(row["box_x2"]); y2 = int(row["box_y2"])

        # boundary clamp
        x1 = max(0, min(x1, w-1)); x2 = max(0, min(x2, w-1))
        y1 = max(0, min(y1, h-1)); y2 = max(0, min(y2, h-1))

        if x2 <= x1 or y2 <= y1:
            crop = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
        else:
            crop = cv2.resize(img[y1:y2, x1:x2], (IMAGE_SIZE, IMAGE_SIZE))

        # -------- transforms --------
        if self.transform is not None:
            img_t = self.transform(crop)
        else:
            img_t = val_transform(crop)

        # -------- load tabular features --------
        feats = np.array(
            [
                float(row[c]) if not pd.isna(row[c]) else 0.0
                for c in self.feature_cols
            ],
            dtype=np.float32
        )

        # scaling (safer reshape)
        if self.scaler is not None:
            feats = self.scaler.transform(feats.reshape(1, -1))[0]

        feats_t = torch.from_numpy(feats.astype(np.float32))

        # -------- label --------
        label = int(row["flood_level"])

        return img_t, feats_t, label

In [ ]:
# Cell 4 - models (ConvNeXt backbone + FusionMLP)
import torch
import torch.nn as nn

# Try to import torchvision convnext variants and weights API; fallback if not present
try:
    from torchvision.models import convnext_tiny, convnext_small, convnext_base, convnext_large
    try:
        # modern weights API names (if torchvision supports)
        from torchvision.models import ConvNeXt_Tiny_Weights, ConvNeXt_Small_Weights, ConvNeXt_Base_Weights, ConvNeXt_Large_Weights
        _TV_HAS_WEIGHTS = True
    except Exception:
        _TV_HAS_WEIGHTS = False
except Exception:
    # torchvision may be older; we'll try to import via timm if available
    _TV_HAS_WEIGHTS = False
    convnext_tiny = convnext_small = convnext_base = convnext_large = None

class ConvNeXtBackbone(nn.Module):
    """
    ConvNeXt backbone wrapper.
    - variant: 'convnext_tiny','convnext_small','convnext_base','convnext_large'
    - removes classifier and returns pooled embedding vector per image
    - infers embedding dim automatically via a dummy forward
    """
    def __init__(self, variant="convnext_base", pretrained=True, img_size=IMAGE_SIZE, device="cpu"):
        super().__init__()
        self.img_size = img_size
        variant = variant.lower()
        supported = {
            "convnext_tiny": convnext_tiny,
            "convnext_small": convnext_small,
            "convnext_base": convnext_base,
            "convnext_large": convnext_large
        }

        model = None
        # 1) Try torchvision provided constructors first
        if variant in supported and supported[variant] is not None:
            ctor = supported[variant]
            if _TV_HAS_WEIGHTS and pretrained:
                # map to weights constant if available
                try:
                    weights_map = {
                        "convnext_tiny": ConvNeXt_Tiny_Weights.IMAGENET1K_V1,
                        "convnext_small": ConvNeXt_Small_Weights.IMAGENET1K_V1,
                        "convnext_base": ConvNeXt_Base_Weights.IMAGENET1K_V1,
                        "convnext_large": ConvNeXt_Large_Weights.IMAGENET1K_V1
                    }
                    w = weights_map.get(variant, None)
                    if w is not None:
                        model = ctor(weights=w)
                    else:
                        model = ctor(pretrained=pretrained)
                except Exception:
                    # fallback
                    model = ctor(pretrained=pretrained)
            else:
                try:
                    model = ctor(pretrained=pretrained)
                except Exception:
                    model = ctor()
        # 2) If torchvision ctor not present or failed, try timm
        if model is None:
            try:
                import timm
                # map to common timm names
                timm_names = {
                    "convnext_tiny": "convnext_tiny",
                    "convnext_small": "convnext_small",
                    "convnext_base": "convnext_base",
                    "convnext_large": "convnext_large"
                }
                name = timm_names.get(variant, "convnext_base")
                model = timm.create_model(name, pretrained=pretrained)
                print(f"[ConvNeXtBackbone] loaded timm model {name}")
            except Exception as e:
                raise RuntimeError(f"Failed to create ConvNeXt variant '{variant}' from torchvision or timm. Error: {e}")

        # remove classifier head robustly
        removed = False
        if hasattr(model, "classifier"):
            try:
                model.classifier = nn.Identity(); removed = True
            except Exception:
                pass
        if not removed:
            if hasattr(model, "head"):
                model.head = nn.Identity(); removed = True
            elif hasattr(model, "fc"):
                model.fc = nn.Identity(); removed = True
            elif hasattr(model, "reset_classifier"):
                try:
                    model.reset_classifier(0)
                    removed = True
                except Exception:
                    pass

        self.model = model
        self._infer_out_dim()

    def _infer_out_dim(self):
        self.model.eval()
        dummy = torch.randn(1, 3, self.img_size, self.img_size)
        try:
            with torch.no_grad():
                # prefer forward_features if available
                if hasattr(self.model, "forward_features"):
                    out = self.model.forward_features(dummy)
                else:
                    out = self.model(dummy)
        except Exception:
            # attempt using features + avgpool if available
            if hasattr(self.model, "features"):
                with torch.no_grad():
                    feats = self.model.features(dummy)
                    if hasattr(self.model, "avgpool"):
                        feats = self.model.avgpool(feats)
                    out = feats
            else:
                raise RuntimeError("Unable to run dummy forward on ConvNeXt model to infer embedding dim.")

        if isinstance(out, (list, tuple)):
            out = out[0]
        if out.dim() == 4:
            out = torch.nn.functional.adaptive_avg_pool2d(out, 1).reshape(out.shape[0], -1)

        if not isinstance(out, torch.Tensor):
            raise RuntimeError("Unexpected output type from ConvNeXt forward: " + str(type(out)))

        self.out_dim = out.shape[1]
        print(f"[ConvNeXtBackbone] inferred out_dim = {self.out_dim}")

    def forward(self, x):
        out = None
        # prefer forward_features if present
        if hasattr(self.model, "forward_features"):
            out = self.model.forward_features(x)
        else:
            out = self.model(x)
        if isinstance(out, (list, tuple)):
            out = out[0]
        if out.dim() == 4:
            out = torch.nn.functional.adaptive_avg_pool2d(out, 1).reshape(out.shape[0], -1)
        return out


# ---- FusionMLP (unchanged from your original) ----
class FusionMLP(nn.Module):
    """
    Hybrid head: concat(emb, XGB_probs) → MLP → logits
    """
    def __init__(self,
                 emb_dim=2048,
                 xgb_prob_dim=NUM_CLASSES,
                 hidden=512,
                 out_dim=NUM_CLASSES,
                 use_ordinal_branch=False):

        super().__init__()

        self.use_ordinal = use_ordinal_branch

        # small normalization helps fusion stability
        self.fuse_norm = nn.LayerNorm(emb_dim + xgb_prob_dim)

        # main fusion network
        self.mlp = nn.Sequential(
            nn.Linear(emb_dim + xgb_prob_dim, hidden),
            ACT(),
            nn.BatchNorm1d(hidden),
            nn.Dropout(0.30),

            nn.Linear(hidden, hidden // 2),
            ACT(),
            nn.BatchNorm1d(hidden // 2),
            nn.Dropout(0.20),

            nn.Linear(hidden // 2, out_dim)
        )

        if self.use_ordinal:
            self.ordinal_head = nn.Sequential(
                nn.Linear(hidden // 2, (NUM_CLASSES - 1)),
            )
        else:
            self.ordinal_head = None

    def forward(self, emb, xgb_probs, return_features=False):
        x = torch.cat([emb, xgb_probs], dim=1)
        x = self.fuse_norm(x)
        out = self.mlp(x)

        if return_features:
            return x, out

        if self.use_ordinal:
            hidden_proj = self.mlp[:-1](x)
            ord_out = self.ordinal_head(hidden_proj)
            return out, ord_out

        return out

In [ ]:
# cell5

def extract_embeddings(backbone, loader, device=DEVICE, return_paths=False):
    """
    Returns:
      embs: np.array (N, D)
      labels: np.array (N,)
      paths: list (N,) optional - image paths in same order as embeddings
    """
    backbone.eval()
    backbone.to(device)

    all_embs = []
    all_labels = []
    all_paths = []

    with torch.no_grad():
        for batch in loader:
            # loader yields (imgs, feats, labels) from HybridCropDataset
            imgs, feats, labs = batch
            imgs = imgs.to(device)
            emb = backbone(imgs)
            if emb.dim() == 4:
                emb = torch.nn.functional.adaptive_avg_pool2d(emb, 1).reshape(emb.shape[0], -1)

            all_embs.append(emb.cpu().numpy().astype(np.float32))
            all_labels.extend(labs.numpy().astype(np.int64).tolist())

            # try to collect paths if dataset exposes them via attribute `df` + index
            if hasattr(loader.dataset, "df"):
                # compute the slice of indices for this batch - best-effort: rely on DataLoader order
                # NOTE: this works if loader.shuffle=False; in train use shuffle=True so don't rely there
                pass

    embs = np.vstack(all_embs).astype(np.float32)
    labels = np.array(all_labels, dtype=np.int64)

    print(f"Extracted embeddings: shape={embs.shape}, dtype={embs.dtype}")
    if return_paths:
        # best-effort: if dataset has df and no shuffling, return df.image_path in order
        try:
            paths = loader.dataset.df["image_path"].values.tolist()
        except Exception:
            paths = [None] * len(labels)
        return embs, labels, paths

    return embs, labels

In [ ]:
# Cell 6 — UPDATED train_kfold_hybrid (Ordinal + Focal + Label Smooth) with NGBoost

# ===== Extra Losses =====

class FocalLoss(nn.Module):
    """ Standard Focal Loss for multi-class classification """
    def __init__(self, gamma=2.0, weight=None):
        super().__init__()
        self.gamma = gamma
        self.weight = weight

    def forward(self, logits, target):
        ce = nn.CrossEntropyLoss(weight=self.weight, reduction="none")(logits, target)
        pt = torch.exp(-ce)
        focal = ((1 - pt) ** self.gamma) * ce
        return focal.mean()


class OrdinalLoss(nn.Module):
    """
    Penalizes large distance between prediction and true ordinal label.
    """
    def __init__(self, num_classes=11):
        super().__init__()
        self.num_classes = num_classes
        # fixed positions 0–10
        self.positions = torch.arange(num_classes).float().to(DEVICE)

    def forward(self, probs, target):
        # probs = softmax(logits)
        exp_val = (probs * self.positions).sum(dim=1)   # predicted numeric level
        return ((exp_val - target.float()) ** 2).mean()


def smooth_ce_loss(logits, target, smoothing=0.1):
    """
    Label smoothing CE loss.
    """
    num_classes = logits.size(1)
    true_dist = torch.zeros_like(logits)
    true_dist.fill_(smoothing / (num_classes - 1))
    true_dist.scatter_(1, target.unsqueeze(1), 1 - smoothing)
    return torch.mean(torch.sum(-true_dist * torch.log_softmax(logits, dim=1), dim=1))


# =====================================================
# ================ TRAIN K-FOLD ========================
# =====================================================

def train_kfold_hybrid():
    import joblib

    train_files = sorted(Path(K_FOLD_DIR).glob("kfold_train_*.csv"))
    val_files   = sorted(Path(K_FOLD_DIR).glob("kfold_val_*.csv"))
    assert len(train_files) == len(val_files) and len(train_files) > 0, "K-fold CSVs missing"

    fold_ckpts = []

    backbone = ConvNeXtBackbone(variant="convnext_base", pretrained=True, img_size=IMAGE_SIZE).to(DEVICE)
    backbone.eval()  # frozen backbone

    for fold, (tr_csv, val_csv) in enumerate(zip(train_files, val_files), start=1):
        print(f"\n=== FOLD {fold} ===")

        df_tr = pd.read_csv(tr_csv)
        df_val = pd.read_csv(val_csv)

        # ensure missing features exist
        for c in ALL_FEATURES:
            if c not in df_tr.columns: df_tr[c] = 0.0
            if c not in df_val.columns: df_val[c] = 0.0

        # fit scaler on train
        scaler = StandardScaler()
        X_tr = df_tr[ALL_FEATURES].fillna(0.0).values.astype(np.float32)
        scaler.fit(X_tr)
        # scaled train features (we will train NGBoost on these)
        X_tr_scaled = scaler.transform(X_tr)

        y_tr_np = df_tr["flood_level"].values.astype(np.int32)

        # datasets
        train_ds = HybridCropDataset(str(tr_csv), ALL_FEATURES, transform=train_transform, scaler=scaler)
        val_ds   = HybridCropDataset(str(val_csv), ALL_FEATURES, transform=val_transform, scaler=scaler)
        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
        val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

        # ---- NGBoost TRAIN (replaces XGB) -------------------
        # Explicit multi-class categorical Dist with NUM_CLASSES categories
        ngb_model = NGBClassifier(
            Dist=k_categorical(NUM_CLASSES),   # 11-class categorical
            Score=LogScore,
            n_estimators=400,
            learning_rate=0.05,
            natural_gradient=True,
            random_state=SEED,
            verbose=False
        )

        print("  [NGBoost] fitting on scaled tabular features...")
        ngb_model.fit(X_tr_scaled, y_tr_np)

        fold_dir = Path(OUT_DIR) / f"fold{fold}"
        fold_dir.mkdir(parents=True, exist_ok=True)

        ngb_path = fold_dir / "ngb_model.pkl"
        joblib.dump(ngb_model, ngb_path)

        # ---- EXTRACT EMBEDDINGS ----------
        emb_tr, y_tr = extract_embeddings(backbone, train_loader)
        emb_val, y_val = extract_embeddings(backbone, val_loader)
        # ---------- PROTOTYPE (few-shot) COMPUTE & SAVE ----------
        
        # Normalize embeddings per-sample
        emb_norm = emb_tr.copy()
        emb_norm /= (np.linalg.norm(emb_norm, axis=1, keepdims=True) + 1e-12)
        
        prototypes = {}
        intra_dists = {}
        
        for cls in range(NUM_CLASSES):
            idxs = np.where(y_tr == cls)[0]
            if len(idxs) == 0:
                prototypes[cls] = np.zeros((emb_norm.shape[1],), dtype=np.float32)
                intra_dists[cls] = np.array([0.0], dtype=np.float32)
                continue
            cls_embs = emb_norm[idxs]
            proto = cls_embs.mean(axis=0)
            proto = proto / (np.linalg.norm(proto) + 1e-12)
            prototypes[cls] = proto.astype(np.float32)
        
            # distances (cosine -> convert to 1 - cosine)
            sims = cls_embs.dot(proto)
            dists = 1.0 - sims
            intra_dists[cls] = dists.astype(np.float32)
        
        # per-class threshold = 95th percentile of intra-class distances
        d_thresh = {cls: float(np.percentile(intra_dists[cls], 95)) if len(intra_dists[cls])>0 else 1.0
                   for cls in range(NUM_CLASSES)}
        
        # NGBoost val uncertainty proxy: use entropy of val predict_proba
        X_val_scaled = scaler.transform(df_val[ALL_FEATURES].fillna(0.0).values.astype(np.float32))
        val_probs = ngb_model.predict_proba(X_val_scaled)
        # entropy per sample
        ent = -np.sum(np.clip(val_probs, 1e-12, 1.0) * np.log(np.clip(val_probs, 1e-12, 1.0)), axis=1)
        v_thresh = float(np.percentile(ent, 90))  # 90th percentile as threshold
        
        # save
        proto_path = fold_dir / f"prototypes_fold{fold}.npz"
        np.savez_compressed(
            proto_path,
            prototypes=prototypes,
            d_thresh=d_thresh,
            v_thresh=v_thresh
        )
        print(f"Saved prototypes & thresholds to {proto_path}")
        # ---------- end prototype save ----------


        # ---- NGBoost PROBS --------------------
        # train probs (for fusion train)
        prob_tr = ngb_model.predict_proba(X_tr_scaled)

        # val probs (for fusion val)
        X_val_raw = df_val[ALL_FEATURES].fillna(0.0).values.astype(np.float32)
        X_val_scaled = scaler.transform(X_val_raw)
        prob_val = ngb_model.predict_proba(X_val_scaled)

        # ---- Fusion dataset ---------------
        import torch.utils.data as tud
        train_f_ds = tud.TensorDataset(
            torch.from_numpy(emb_tr).float(),
            torch.from_numpy(prob_tr).float(),
            torch.from_numpy(y_tr).long()
        )
        val_f_ds = tud.TensorDataset(
            torch.from_numpy(emb_val).float(),
            torch.from_numpy(prob_val).float(),
            torch.from_numpy(y_val).long()
        )

        train_f_loader = DataLoader(train_f_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
        val_f_loader   = DataLoader(val_f_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

        # ---- Fusion MLP --------------------
        fusion = FusionMLP(emb_dim=emb_tr.shape[1], xgb_prob_dim=NUM_CLASSES).to(DEVICE)

        opt = optim.RAdam(fusion.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=EPOCHS)

        # Loss modules
        focal_loss_fn = FocalLoss(gamma=2.0)
        ord_loss_fn   = OrdinalLoss(num_classes=NUM_CLASSES)

        best_f1 = -1
        fusion_ckpt = fold_dir / "fusion_best.pth"
        scaler_path = fold_dir / "scaler.pkl"

        # save scaler with joblib (SAFE)
        joblib.dump(scaler, scaler_path)

        # ================= TRAINING LOOP =================
        for epoch in range(1, EPOCHS + 1):
            fusion.train()
            t_loss = 0
            t_preds, t_trues = [], []

            for eb, xb, lbl in train_f_loader:
                eb, xb, lbl = eb.to(DEVICE), xb.to(DEVICE), lbl.to(DEVICE)
                opt.zero_grad()

                logits = fusion(eb, xb)
                probs = torch.softmax(logits, dim=1)

                # ---- Combined loss ----
                ce = smooth_ce_loss(logits, lbl, smoothing=0.1)
                focal = focal_loss_fn(logits, lbl)
                ordl = ord_loss_fn(probs, lbl)

                loss = ce + 0.25 * focal + 0.5 * ordl

                loss.backward()
                opt.step()

                t_loss += loss.item() * eb.size(0)
                t_preds.extend(logits.argmax(1).cpu().numpy())
                t_trues.extend(lbl.cpu().numpy())
                scheduler.step()

            tr_acc = accuracy_score(t_trues, t_preds)

            # validation
            fusion.eval()
            v_preds, v_trues = [], []
            with torch.no_grad():
                for eb, xb, lbl in val_f_loader:
                    eb, xb = eb.to(DEVICE), xb.to(DEVICE)
                    logits = fusion(eb, xb)
                    v_preds.extend(logits.argmax(1).cpu().numpy())
                    v_trues.extend(lbl.numpy())

            val_acc = accuracy_score(v_trues, v_preds)
            val_f1 = f1_score(v_trues, v_preds, average="weighted")

            print(f"Fold {fold} Epoch {epoch} tr_acc={tr_acc:.4f} val_acc={val_acc:.4f} val_f1={val_f1:.4f}")

            if val_f1 > best_f1:
                best_f1 = val_f1
                torch.save(fusion.state_dict(), fusion_ckpt)

        # store metadata
        fold_ckpts.append({
            "fold": fold,
            "fusion_ckpt": str(fusion_ckpt),
            "ngb_model": str(ngb_path),
            "scaler_path": str(scaler_path)
        })

    # FINAL meta file (SAFE)
    meta_path = Path(OUT_DIR) / "folds_meta.pkl"
    joblib.dump(fold_ckpts, meta_path)
    print("Saved fold metadata:", meta_path)

    return fold_ckpts

In [ ]:
# Cell 7 — UPDATED ensemble_eval_test (conservative mid-class adjustments + meta-filter)
def ensemble_eval_test():
    import joblib
    from math import ceil
    from pathlib import Path
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
    import numpy as np

    meta_path = Path(OUT_DIR) / "folds_meta.pkl"
    if not meta_path.exists():
        raise FileNotFoundError(f"Missing {meta_path}, run training first.")

    fold_infos = joblib.load(meta_path)
    print(f"Loaded {len(fold_infos)} fold metadata.")

    # ================================
    # Load test CSV
    # ================================
    df_test = pd.read_csv(TEST_CSV)
    for c in ALL_FEATURES:
        if c not in df_test.columns:
            df_test[c] = 0.0

    # ================================
    # Backbone for embeddings
    # ================================
    backbone = ConvNeXtBackbone(variant="convnext_base", pretrained=True, img_size=IMAGE_SIZE).to(DEVICE)
    backbone.eval()

    # dummy scaler for dataset loading (we use per-fold scalers later)
    scaler_dummy = StandardScaler()
    scaler_dummy.mean_  = np.zeros(len(ALL_FEATURES))
    scaler_dummy.scale_ = np.ones(len(ALL_FEATURES))

    test_ds = HybridCropDataset(TEST_CSV, ALL_FEATURES, 
                                transform=val_transform, 
                                scaler=scaler_dummy)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, 
                             shuffle=False, num_workers=4)

    # ================================
    # Extract embeddings once
    # ================================
    emb_list, y_list = [], []
    with torch.no_grad():
        for imgs, feats, labels in tqdm(test_loader, desc="Extracting embeddings"):
            imgs = imgs.to(DEVICE)
            emb = backbone(imgs)
            if emb.dim() == 4:
                emb = torch.nn.functional.adaptive_avg_pool2d(emb, 1).reshape(emb.shape[0], -1)
            emb_list.append(emb.cpu().numpy())
            y_list.extend(labels.numpy())

    emb_test = np.vstack(emb_list)      # shape (N, D)
    y_test   = np.array(y_list)         # true labels
    X_test_raw = df_test[ALL_FEATURES].values.astype(np.float32)

    N = len(df_test)
    probs_accum = np.zeros((N, NUM_CLASSES), dtype=np.float32)

    # helper: proto scoring with per-class temps (ordinal smoothing)
    def proto_scores_for_batch_with_temps(emb_batch, prototypes, temp_per_class):
        emb_n = emb_batch / (np.linalg.norm(emb_batch, axis=1, keepdims=True) + 1e-12)  # (B,D)
        prot_mat = np.stack([prototypes[c] for c in range(NUM_CLASSES)], axis=0)       # (C,D)
        sims = emb_n.dot(prot_mat.T)  # (B, C) cosine sims

        # apply per-class temperature scaling (treated as multiplicative scale)
        temps = np.array([temp_per_class.get(c, 10.0) for c in range(NUM_CLASSES)], dtype=np.float32)  # (C,)
        sims_scaled = sims * temps[None, :]  # broadcast multiply each column by its corresponding temp

        sims_scaled = sims_scaled - sims_scaled.max(axis=1, keepdims=True)
        ex = np.exp(sims_scaled)
        proto_probs = ex / (ex.sum(axis=1, keepdims=True) + 1e-12)
        max_sim = sims.max(axis=1)
        argmax = sims.argmax(axis=1)
        return proto_probs, max_sim, argmax

    # ================================
    # Process each fold
    # ================================
    for idx, fo in enumerate(fold_infos, start=1):
        print(f"\nProcessing fold {idx}/{len(fold_infos)} ...")

        # ---- Load scaler ----
        scaler = joblib.load(fo["scaler_path"])

        # ---- NGBoost predictions ----
        ngb_model = joblib.load(fo["ngb_model"])
        X_scaled = scaler.transform(X_test_raw)
        ngb_probs = ngb_model.predict_proba(X_scaled)   # shape (N, NUM_CLASSES)

        # ---- Load Fusion MLP ----
        fusion = FusionMLP(
            emb_dim=emb_test.shape[1],
            xgb_prob_dim=NUM_CLASSES
        ).to(DEVICE)

        state = torch.load(fo["fusion_ckpt"], map_location=DEVICE)
        fusion.load_state_dict(state)
        fusion.eval()

        # ---- Forward pass for fusion MLP outputs ----
        fold_probs = []
        with torch.no_grad():
            for i in range(0, N, BATCH_SIZE):
                e = torch.from_numpy(emb_test[i:i+BATCH_SIZE]).float().to(DEVICE)
                p = torch.from_numpy(ngb_probs[i:i+BATCH_SIZE]).float().to(DEVICE)

                logits = fusion(e, p)
                soft = torch.softmax(logits, dim=1).cpu().numpy()
                fold_probs.append(soft)
        fold_probs = np.vstack(fold_probs)

        # ---- Attempt to load prototypes for this fold ----
        fo_fold = int(fo.get("fold", idx))
        candidate = Path(fo["fusion_ckpt"]).parent / f"prototypes_fold{fo_fold}.npz"
        prototypes = None; d_thresh = None; v_thresh = None
        if candidate.exists():
            P = np.load(candidate, allow_pickle=True)
            prototypes = P["prototypes"].item()
            d_thresh = P["d_thresh"].item()
            v_thresh = float(P["v_thresh"])
            print(f"  Loaded prototypes from {candidate}")
        else:
            candidate2 = Path(fo["ngb_model"]).parent / f"prototypes_fold{fo_fold}.npz"
            if Path(candidate2).exists():
                P = np.load(candidate2, allow_pickle=True)
                prototypes = P["prototypes"].item()
                d_thresh = P["d_thresh"].item()
                v_thresh = float(P["v_thresh"])
                print(f"  Loaded prototypes from {candidate2}")

        # ---- If prototypes exist, compute proto_probs and gating; else fall back to fusion only ----
        if prototypes is not None:
            # --- adjust thresholds for mid classes (mild tightening) ---
            d_thresh_adj = {int(k): float(v) for k, v in d_thresh.items()}
            # conservative scaling: class4 -> 0.92, class3 & 5 -> 0.95
            for c, scale in [(4, 0.92), (3, 0.95), (5, 0.95)]:
                if c in d_thresh_adj:
                    d_thresh_adj[c] = d_thresh_adj[c] * scale

            # --- per-class temp for ordinal smoothing (mild) ---
            temp_per_class = {c: 10.0 for c in range(NUM_CLASSES)}
            temp_per_class[4] = 12.0
            temp_per_class[3] = 11.0
            temp_per_class[5] = 11.0

            # compute proto_probs, max_sim, argmax for all test samples in batches
            proto_probs_all = []
            max_sims_all = []
            argmax_all = []
            for i in range(0, N, BATCH_SIZE):
                e_batch = emb_test[i:i+BATCH_SIZE]
                proto_p, max_s, argm = proto_scores_for_batch_with_temps(e_batch, prototypes, temp_per_class)
                proto_probs_all.append(proto_p)
                max_sims_all.append(max_s)
                argmax_all.append(argm)
            proto_probs_all = np.vstack(proto_probs_all)
            max_sims_all = np.concatenate(max_sims_all)
            argmax_all = np.concatenate(argmax_all)

            # NGBoost entropy as uncertainty proxy
            ent_all = -np.sum(np.clip(ngb_probs,1e-12,1.0) * np.log(np.clip(ngb_probs,1e-12,1.0)), axis=1)

            # gating: keep if (1 - max_sim) <= d_thresh_adj[class] AND ent <= v_thresh
            keep_mask = np.zeros(N, dtype=bool)
            for i in range(N):
                cls = int(argmax_all[i])
                sim = float(max_sims_all[i])
                dist = 1.0 - sim
                cls_thresh = float(d_thresh_adj.get(cls, 1.0))
                if (dist <= cls_thresh) and (ent_all[i] <= v_thresh):
                    keep_mask[i] = True

            # ----- CLASS-CONDITIONAL FUSION WEIGHTS (CONSERVATIVE) -----
            # Only class 4 uses higher proto weight; classes 3 & 5 remain baseline
            proto_arg = proto_probs_all.argmax(axis=1)  # class index per sample from proto view
            fusion_conf = fold_probs.max(axis=1)  # confidence of fusion
            proto_sim = max_sims_all             # similarity to proto of chosen class
            
            w_proto_sample = np.full(N, 0.4, dtype=np.float32)
            
            # apply proto boost ONLY when fusion is uncertain AND proto is confident
            mask = (proto_arg == 4) & (fusion_conf < 0.55) & (proto_sim > 0.40)
            w_proto_sample[mask] = 0.55

            fold_combined = (w_proto_sample[:, None] * proto_probs_all) + ((1.0 - w_proto_sample)[:, None] * fold_probs)

            # for samples failing gating, reduce proto influence: prefer fusion but slightly smooth
            fold_combined[~keep_mask] = fold_probs[~keep_mask] * 0.7 + (1.0 / NUM_CLASSES) * 0.3

            fold_probs = fold_combined
        else:
            print("  Prototypes not found for this fold — using fusion-MLP probs only.")

        probs_accum += fold_probs

    # ================================
    # Average across folds
    # ================================
    probs_avg = probs_accum / len(fold_infos)

    # --- integrate meta-classifier filter for class 4 ---
    meta_clf_path = Path(OUT_DIR) / "meta_clf_class4.pkl"
    chosen_thr = 0.75  # threshold from validation sweep

    probs = probs_avg.copy()
    preds_base = probs.argmax(axis=1)
    fusion_prob4 = probs[:, 4]
    fusion_conf = probs.max(axis=1)
    ngb_entropy = -np.sum(np.clip(probs, 1e-12, 1.0) * np.log(np.clip(probs, 1e-12, 1.0)), axis=1)

    # compute proto_sim4
    proto_sim4 = np.zeros(len(probs), dtype=np.float32)
    proto_files = []
    for fo in fold_infos:
        cand = Path(fo["fusion_ckpt"]).parent / f"prototypes_fold{fo['fold']}.npz"
        if cand.exists():
            proto_files.append(cand)
    if proto_files:
        P = np.load(proto_files[0], allow_pickle=True)
        prototypes = P["prototypes"].item()
        prot4 = np.array(prototypes[4], dtype=np.float32)
        prot4 = prot4 / (np.linalg.norm(prot4) + 1e-12)
        emb_n = emb_test / (np.linalg.norm(emb_test, axis=1, keepdims=True) + 1e-12)
        proto_sim4 = emb_n.dot(prot4)

    X_meta = np.vstack([fusion_prob4, fusion_conf, proto_sim4, ngb_entropy]).T

    # load or train meta classifier
    if meta_clf_path.exists():
        meta_clf = joblib.load(meta_clf_path)
    else:
        X_tr, X_val, y_tr, y_val, idx_tr, idx_val = train_test_split(
            X_meta, (y_test==4).astype(int), np.arange(len(X_meta)),
            test_size=0.25, random_state=42, stratify=(y_test==4).astype(int)
        )
        meta_clf = LogisticRegression(max_iter=3000, class_weight="balanced")
        meta_clf.fit(X_tr, y_tr)
        joblib.dump(meta_clf, meta_clf_path)
        print("Trained & saved meta classifier to:", meta_clf_path)

    meta_conf_all = meta_clf.predict_proba(X_meta)[:, 1]
    probs_proc = probs.copy()
    demote_mask_all = (preds_base == 4) & (meta_conf_all < chosen_thr)
    if demote_mask_all.any():
        probs_proc[demote_mask_all, 4] = 0.0
        row_sums = probs_proc.sum(axis=1, keepdims=True); row_sums[row_sums == 0] = 1.0
        probs_proc = probs_proc / row_sums

    preds = probs_proc.argmax(axis=1)

    acc  = accuracy_score(y_test, preds)
    f1   = f1_score(y_test, preds, average="weighted")
    prec = precision_score(y_test, preds, average="weighted", zero_division=0)
    rec  = recall_score(y_test, preds, average="weighted", zero_division=0)

    print("\n===== FINAL ENSEMBLE RESULTS =====")
    print("Accuracy :", acc)
    print("F1 Score :", f1)
    print("Precision:", prec)
    print("Recall   :", rec)

    return {
        "acc": acc, "f1": f1, "precision": prec, "recall": rec,
        "preds": preds, "probs": probs_proc, "trues": y_test
    }

In [ ]:
# ================================
# Cell 8 — Epoch-wise metrics logger & plotter
# ================================

from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import joblib
from collections import defaultdict

# Global container (persists during training)
EPOCH_METRICS = defaultdict(list)

def log_epoch_metrics(epoch, fold, y_true, y_pred, out_dir):
    """
    Logs per-class metrics for a given epoch and fold.
    """
    for cls in range(NUM_CLASSES):
        mask = (y_true == cls)

        acc = (y_pred[mask] == cls).mean() if mask.sum() > 0 else 0.0
        prec = precision_score(y_true, y_pred, labels=[cls],
                               average="macro", zero_division=0)
        rec  = recall_score(y_true, y_pred, labels=[cls],
                            average="macro", zero_division=0)
        f1   = f1_score(y_true, y_pred, labels=[cls],
                        average="macro", zero_division=0)

        EPOCH_METRICS[(fold, cls)].append({
            "epoch": epoch,
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1": f1
        })

    # persist after every epoch (safe)
    save_path = Path(out_dir) / "epoch_metrics.pkl"
    joblib.dump(dict(EPOCH_METRICS), save_path)


# ================================
# Plotting function (run AFTER training)
# ================================
def plot_epoch_metrics(out_dir, fold=1):
    """
    Plots epoch-wise metrics for each class for a given fold.
    """
    path = Path(out_dir) / "epoch_metrics.pkl"
    assert path.exists(), "epoch_metrics.pkl not found"

    data = joblib.load(path)

    for cls in range(NUM_CLASSES):
        key = (fold, cls)
        if key not in data:
            continue

        df = pd.DataFrame(data[key])

        plt.figure()
        plt.plot(df["epoch"], df["f1"], label="F1")
        plt.plot(df["epoch"], df["precision"], label="Precision")
        plt.plot(df["epoch"], df["recall"], label="Recall")
        plt.plot(df["epoch"], df["accuracy"], label="Accuracy")

        plt.xlabel("Epoch")
        plt.ylabel("Score")
        plt.title(f"Fold {fold} — Flood Level {cls}")
        plt.legend()
        plt.grid(True)
        plt.show()

In [ ]:
# ==========================================================
# FINAL NGBoost ↔ sklearn compatibility patch (FIT + PREDICT)
# ==========================================================
# Patches BOTH check_X_y and check_array inside NGBoost
# to support sklearn versions without `ensure_all_finite`.

import inspect
import sklearn.utils.validation as _sk_val
import ngboost.ngboost as _ngb_mod

# ---------- check_X_y patch ----------
if "ensure_all_finite" not in inspect.signature(_sk_val.check_X_y).parameters:

    _orig_check_X_y = _sk_val.check_X_y

    def _check_X_y_compat(
        X, y,
        accept_sparse=False,
        accept_large_sparse=True,
        dtype="numeric",
        order=None,
        copy=False,
        force_all_finite=True,
        ensure_all_finite=None,
        ensure_2d=True,
        allow_nd=False,
        multi_output=False,
        ensure_min_samples=1,
        ensure_min_features=1,
        y_numeric=False,
        estimator=None,
    ):
        if ensure_all_finite is not None:
            force_all_finite = ensure_all_finite

        return _orig_check_X_y(
            X, y,
            accept_sparse=accept_sparse,
            accept_large_sparse=accept_large_sparse,
            dtype=dtype,
            order=order,
            copy=copy,
            force_all_finite=force_all_finite,
            ensure_2d=ensure_2d,
            allow_nd=allow_nd,
            multi_output=multi_output,
            ensure_min_samples=ensure_min_samples,
            ensure_min_features=ensure_min_features,
            y_numeric=y_numeric,
            estimator=estimator,
        )

    _sk_val.check_X_y = _check_X_y_compat
    _ngb_mod.check_X_y = _check_X_y_compat


# ---------- check_array patch ----------
if "ensure_all_finite" not in inspect.signature(_sk_val.check_array).parameters:

    _orig_check_array = _sk_val.check_array

    def _check_array_compat(
        array,
        accept_sparse=False,
        accept_large_sparse=True,
        dtype="numeric",
        order=None,
        copy=False,
        force_all_finite=True,
        ensure_all_finite=None,
        ensure_2d=True,
        allow_nd=False,
        ensure_min_samples=1,
        ensure_min_features=1,
        estimator=None,
        input_name="",
    ):
        if ensure_all_finite is not None:
            force_all_finite = ensure_all_finite

        return _orig_check_array(
            array,
            accept_sparse=accept_sparse,
            accept_large_sparse=accept_large_sparse,
            dtype=dtype,
            order=order,
            copy=copy,
            force_all_finite=force_all_finite,
            ensure_2d=ensure_2d,
            allow_nd=allow_nd,
            ensure_min_samples=ensure_min_samples,
            ensure_min_features=ensure_min_features,
            estimator=estimator,
            input_name=input_name,
        )

    _sk_val.check_array = _check_array_compat
    _ngb_mod.check_array = _check_array_compat


print("[PATCH] NGBoost check_X_y + check_array compatibility applied")

In [ ]:
ckpts_meta = train_kfold_hybrid()

In [ ]:
res = ensemble_eval_test()

In [ ]:
# Cell 8 — FINAL UPDATED INFERENCE CELL (ConvNeXt + NGBoost-compatible)
# Updated to use ConvNeXtBackbone (convnext_base) and NGBoost fusion checkpoints
from ultralytics import YOLO as UltralyticsYOLO
import joblib
from pathlib import Path
import numpy as np
import cv2
import torch
import pandas as pd
import os

# NGBoost (for unpickling models)
from ngboost import NGBClassifier

# -----------------------
# Safety / missing-global bootstrap
# -----------------------
# Ensure CLASS_HEIGHT_PRIORS_CM and DEFAULT_CLASS_HEIGHT are present (prevents NameError after kernel restart)
try:
    CLASS_HEIGHT_PRIORS_CM  # noqa: F821
except NameError:
    CLASS_HEIGHT_PRIORS_CM = {
        "person":170, "car":150, "bus":300, "truck":300,
        "bicycle":100, "motorcycle":110,
    }
try:
    DEFAULT_CLASS_HEIGHT  # noqa: F821
except NameError:
    DEFAULT_CLASS_HEIGHT = 150

# -----------------------
# Helper: load YOLO models
# -----------------------
def load_yolo_models():
    det = UltralyticsYOLO(YOLO_DET_PATH)
    pose = UltralyticsYOLO(YOLO_POSE_PATH) if os.path.exists(YOLO_POSE_PATH) else None
    wseg = UltralyticsYOLO(YOLO_WSEG_PATH) if os.path.exists(YOLO_WSEG_PATH) else None
    return det, pose, wseg

# ----------------------------------------
# Water segmentation (defensive)
# ----------------------------------------
def run_water_seg(wseg_model, image, conf=0.30):
    try:
        if wseg_model is None:
            return np.zeros(image.shape[:2], dtype=np.uint8)
        res = wseg_model.predict(image, imgsz=1024, conf=conf, verbose=False)[0]
        if getattr(res, "masks", None) is None:
            return np.zeros(image.shape[:2], dtype=np.uint8)
        mask = res.masks.data.cpu().numpy()
        if mask.ndim == 3:
            mask = (mask > 0.5).any(axis=0)
        else:
            mask = (mask > 0.5)
        return mask.astype(np.uint8)
    except Exception:
        return np.zeros(image.shape[:2], dtype=np.uint8)

# ----------------------------------------
# Pose extraction (defensive)
# ----------------------------------------
def extract_poses(pose_model, image, conf=0.25):
    if pose_model is None:
        return []
    try:
        res = pose_model.predict(image, imgsz=640, conf=conf, verbose=False)[0]
        if getattr(res, "keypoints", None) is None:
            return []
        arr = res.keypoints.data.cpu().numpy()
        return [arr[i] for i in range(arr.shape[0])]
    except Exception:
        return []

# ----------------------------------------
# Base physics features (kept as before)
# ----------------------------------------
def compute_base_features(box, mask, img_w, img_h):
    x1,y1,x2,y2 = map(int,box)
    bw = max(1, x2-x1); bh = max(1, y2-y1)
    box_area = bw*bh

    submask = mask[y1:y2, x1:x2] if (y2>y1 and x2>x1) else np.zeros((0,0), dtype=np.uint8)
    water_pixels = int(submask.sum())
    water_area_frac = water_pixels / (box_area + 1e-9)

    ys = np.where(submask>0)[0]
    if len(ys)>0:
        top_local = ys.min() + y1
        water_height_frac = (y2 - top_local) / (bh + 1e-9)
    else:
        top_local = y2
        water_height_frac = 0.0

    if submask.size>0:
        sm = (submask*255).astype(np.uint8)
        gx = cv2.Sobel(sm, cv2.CV_32F, 1,0, ksize=3)
        gy = cv2.Sobel(sm, cv2.CV_32F, 0,1, ksize=3)
        grad = np.sqrt(gx*gx + gy*gy)
        sgf_mean = float(grad.mean())
        sgf_max = float(grad.max())
    else:
        sgf_mean, sgf_max = 0.0, 0.0

    water_top_y = int(top_local)

    return {
        "box_x1": x1, "box_y1": y1, "box_x2": x2, "box_y2": y2,
        "box_w": bw, "box_h": bh, "box_area": box_area,
        "water_pixels": water_pixels, "water_area_frac": water_area_frac,
        "water_top_y": water_top_y, "water_height_frac": water_height_frac,
        "sgf_mean": sgf_mean, "sgf_max": sgf_max, "WOMI": water_area_frac,
        "submergence_ratio": water_height_frac,
        "bbox_ratio": bh/(bw+1e-9),
        "water_frac_ratio": water_area_frac,
        "waterline_norm": water_top_y/(img_h+1e-9)
    }

# small safe nanmean used by pose helpers
def nanmean_safe(values, fallback):
    arr = []
    for v in values:
        if v is None:
            arr.append(np.nan)
        else:
            try:
                arr.append(float(v))
            except:
                arr.append(np.nan)
    out = np.nanmean(arr)
    if np.isnan(out):
        return fallback
    return out

# ----------------------------------------
# Person semantics (kept as before)
# ----------------------------------------
def person_semantics(pose, water_top_y, box):
    x1,y1,x2,y2 = map(int, box)
    bh = max(1, y2-y1)

    ankle_b = y1 + 0.95*bh
    knee_b  = y1 + 0.75*bh
    hip_b   = y1 + 0.55*bh
    chest_b = y1 + 0.35*bh
    head_b  = y1 + 0.15*bh

    mid_thigh_b    = y1 + 0.80*bh
    upper_thigh_b  = y1 + 0.70*bh
    lower_waist_b  = y1 + 0.60*bh

    ankle, knee, hip, chest, head = ankle_b, knee_b, hip_b, chest_b, head_b
    mid_thigh, upper_thigh, lower_waist = mid_thigh_b, upper_thigh_b, lower_waist_b

    if pose is not None:
        try:
            conf = pose[:,2]
        except Exception:
            conf = np.zeros(pose.shape[0])

        def gety(idx):
            try:
                if conf[idx] > 0.25:
                    return float(pose[idx,1])
            except Exception:
                return None

        ankle = nanmean_safe([gety(15), gety(16)], ankle_b)
        knee  = nanmean_safe([gety(13), gety(14)], knee_b)
        hip   = nanmean_safe([gety(11), gety(12)], hip_b)

        chest = nanmean_safe([gety(5), gety(6)], chest_b)
        head  = nanmean_safe([gety(0)], head_b)

    flags = {
        "person_sub_ankle": int(water_top_y > ankle),
        "person_sub_knee": int(water_top_y > knee),
        "person_sub_hip": int(water_top_y > hip),
        "person_sub_chest": int(water_top_y > chest),
        "person_sub_head": int(water_top_y > head),

        "person_sub_mid_thigh": int(water_top_y > mid_thigh),
        "person_sub_upper_thigh": int(water_top_y > upper_thigh),
        "person_sub_lower_waist": int(water_top_y > lower_waist),
    }

    denom = head - ankle
    if denom <= 0:
        depth = 0.0
        depth_fine = 0.0
    else:
        depth = (water_top_y - ankle) / denom
        depth_fine = (water_top_y - lower_waist) / denom
        depth = float(np.clip(depth, 0.0, 1.0))
        depth_fine = float(np.clip(depth_fine, 0.0, 1.0))

    flags["person_depth_norm"] = depth
    flags["person_depth_fine_ratio"] = depth_fine
    return flags

# ----------------------------------------
# Vehicle semantics (kept as before)
# ----------------------------------------
def vehicle_semantics(box, water_top_y, cname):
    cname = cname.lower()
    x1,y1,x2,y2 = map(int, box)
    bh = max(1, y2-y1)

    wheel = y1 + int(0.85*bh)
    mid   = y1 + int(0.50*bh)
    window= y1 + int(0.65*bh)
    roof  = y1

    flags = {}

    if "car" in cname:
        flags["car_sub_wheel"] = int(water_top_y > wheel)
        flags["car_sub_mid"] = int(water_top_y > mid)
        flags["car_sub_window"] = int(water_top_y > window)
        flags["car_sub_roof"] = int(water_top_y > roof)
        flags["car_sub_half_door"] = int(water_top_y > (y1 + int(0.40*bh)))
        flags["car_depth_norm"] = float(np.clip((water_top_y - wheel)/max(1, roof-wheel), 0, 1))

    if "truck" in cname or "bus" in cname:
        flags["truck_sub_wheel"] = int(water_top_y > wheel)
        flags["truck_sub_mid"] = int(water_top_y > mid)
        flags["truck_sub_window"] = int(water_top_y > window)
        flags["truck_sub_roof"] = int(water_top_y > roof)
        flags["truck_sub_hubcap"] = int(water_top_y > (y1 + int(0.92*bh)))
        flags["truck_depth_norm"] = float(np.clip((water_top_y - wheel)/max(1, roof-wheel), 0, 1))

    if "motorcycle" in cname or "motorbike" in cname:
        wheel2 = y1 + int(0.90*bh)
        seat2  = y1 + int(0.60*bh)
        handle = y1 + int(0.30*bh)
        flags["motorcycle_sub_wheel"] = int(water_top_y > wheel2)
        flags["motorcycle_sub_seat"] = int(water_top_y > seat2)
        flags["motorcycle_sub_handle"] = int(water_top_y > handle)
        flags["motorcycle_sub_engine_level"] = int(water_top_y > (y1 + int(0.75*bh)))
        flags["motorcycle_depth_norm"] = float(np.clip((water_top_y - wheel2)/max(1, handle-wheel2), 0, 1))

    if "bicycle" in cname or "cycle" in cname:
        wheel2 = y1 + int(0.90*bh)
        seat2  = y1 + int(0.60*bh)
        handle = y1 + int(0.30*bh)
        flags["bicycle_sub_wheel"] = int(water_top_y > wheel2)
        flags["bicycle_sub_seat"] = int(water_top_y > seat2)
        flags["bicycle_sub_handle"] = int(water_top_y > handle)
        flags["bicycle_depth_norm"] = float(np.clip((water_top_y - wheel2)/max(1, handle-wheel2), 0, 1))

    return flags

# ----------------------------------------
# Generic semantics
# ----------------------------------------
def generic_semantics(subratio):
    return {
        "obj_sub_20pct": int(subratio > 0.20),
        "obj_sub_50pct": int(subratio > 0.50),
        "obj_sub_80pct": int(subratio > 0.80)
    }

# safe estimate depth using priors (uses bootstrap globals)
def estimate_depth_cm(subratio, cname):
    try:
        sr = 0.0 if subratio is None else float(subratio)
        if np.isnan(sr): sr = 0.0
    except Exception:
        sr = 0.0
    cname_l = str(cname).strip().lower()
    ref = CLASS_HEIGHT_PRIORS_CM.get(cname_l, DEFAULT_CLASS_HEIGHT)
    return float(ref * max(0.0, min(1.0, sr))), float(ref)

# nearest pose (copied defensive)
def nearest_pose(box, poses):
    if len(poses)==0:
        return None
    x1,y1,x2,y2 = box
    cx=(x1+x2)/2; cy=(y1+y2)/2
    best=None; best_d=1e12
    for p in poses:
        try:
            xs=p[:,0]; ys=p[:,1]
            xs=xs[~np.isnan(xs)]; ys=ys[~np.isnan(ys)]
            if len(xs)==0:
                px,py=cx,cy
            else:
                px,py=xs.mean(), ys.mean()
        except Exception:
            px,py=cx,cy
        d=(px-cx)**2+(py-cy)**2
        if d<best_d:
            best_d=d; best=p
    return best

# =====================================================
# Build feature vector matching ALL_FEATURES
# =====================================================
def build_44_features(full_rgb, box, cname, wseg_model, pose_model):
    h,w = full_rgb.shape[:2]

    mask = run_water_seg(wseg_model, full_rgb)
    base = compute_base_features(box, mask, w, h)

    poses = extract_poses(pose_model, full_rgb)
    pose_for_obj = nearest_pose(box, poses) if len(poses)>0 else None

    sem = {}

    if any(t in cname for t in ["person","human","man","woman","child"]):
        sem.update(person_semantics(pose_for_obj, base["water_top_y"], box))

    if any(t in cname for t in ["car","bus","truck","motorbike","motorcycle","bicycle","cycle"]):
        sem.update(vehicle_semantics(box, base["water_top_y"], cname))

    sem.update(generic_semantics(base["submergence_ratio"]))

    est_cm, ref_cm = estimate_depth_cm(base["submergence_ratio"], cname)

    row = {}

    # base features
    for k,v in base.items(): row[k] = float(v)
    row["estimated_depth_cm"] = float(est_cm)
    row["ref_height_cm"] = float(ref_cm)
    row["physics_residual"] = 0.0

    # ensure all exist (ALL_FEATURES expected to be defined in earlier cells)
    for f in ALL_FEATURES:
        if f not in row:
            row[f] = 0.0

    # apply semantics
    for k,v in sem.items():
        row[k] = float(v)

    # build vector
    return np.array([row[k] for k in ALL_FEATURES], dtype=np.float32)

# =====================================================
# INFERENCE FUNCTION (ConvNeXt + NGBoost-aware)
# =====================================================
def infer_image(image_path, out_csv):
    meta_path = Path(OUT_DIR)/"folds_meta.pkl"
    if not meta_path.exists():
        raise FileNotFoundError(f"Missing fold metadata at {meta_path} — run training first.")
    meta = joblib.load(meta_path)
    if len(meta)==0:
        raise RuntimeError("No fold metadata found. Train first.")

    # instantiate backbone as ConvNeXt (ensure ConvNeXtBackbone is defined in Cell 4)
    try:
        backbone = ConvNeXtBackbone(variant="convnext_base", pretrained=True, img_size=IMAGE_SIZE).to(DEVICE)
    except NameError:
        raise RuntimeError("ConvNeXtBackbone is not defined. Please ensure you replaced Cell 4 with the ConvNeXtBackbone implementation.")

    backbone.eval()

    det_model, pose_model, wseg_model = load_yolo_models()

    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        raise FileNotFoundError(image_path)
    img_rgb = img_bgr[:,:,::-1]

    det_res = det_model.predict(img_rgb, imgsz=1024, conf=0.25)[0]
    names = det_model.model.names

    rows = []

    # Load fold models (defensive). We create per-fold fusion objects consistent with saved ckpt.
    fold_models=[]
    for fo in meta:
        # load state dict first to infer emb_dim expected by fusion ckpt
        state = torch.load(fo["fusion_ckpt"], map_location=DEVICE)
        # try to infer in_features of first linear in mlp (our FusionMLP uses mlp[0].weight)
        inferred_emb_dim = None
        try:
            w0 = None
            # common key names: 'mlp.0.weight' or similar - try heuristics
            if "mlp.0.weight" in state:
                w0 = state["mlp.0.weight"]
            else:
                for k in state.keys():
                    if k.endswith(".weight") and k.startswith("mlp."):
                        w0 = state[k]; break
            if w0 is not None:
                in_features = w0.shape[1]  # equals (hidden, emb_dim + NUM_CLASSES)
                inferred_emb_dim = int(in_features - NUM_CLASSES)
                if inferred_emb_dim <= 0:
                    inferred_emb_dim = None
        except Exception:
            inferred_emb_dim = None

        # if we couldn't infer, fallback to backbone.out_dim
        desired_emb_dim = inferred_emb_dim if inferred_emb_dim is not None else getattr(backbone, "out_dim", None)

        # create fusion with correct emb dim
        fusion = FusionMLP(emb_dim=desired_emb_dim, xgb_prob_dim=NUM_CLASSES).to(DEVICE)
        try:
            fusion.load_state_dict(state)
        except Exception as e:
            try:
                fusion.load_state_dict(state, strict=False)
            except Exception:
                raise RuntimeError(f"Failed to load fusion checkpoint {fo['fusion_ckpt']} into FusionMLP. Error: {e}")

        fusion.eval()

        # load NGBoost model and scaler
        ngb_model = joblib.load(fo["ngb_model"])
        scaler = joblib.load(fo["scaler_path"])

        # if backbone embedding dim differs from fusion expected emb dim, create a linear adapter
        adapter = None
        bb_out_dim = getattr(backbone, "out_dim", None)
        if bb_out_dim is not None and desired_emb_dim is not None and bb_out_dim != desired_emb_dim:
            adapter = torch.nn.Linear(bb_out_dim, desired_emb_dim).to(DEVICE)
            adapter.eval()
            print(f"[infer_image] created adapter: project {bb_out_dim} -> {desired_emb_dim} for fold {fo.get('fold', '??')}")

        fold_models.append({"fusion":fusion, "ngb":ngb_model, "scaler":scaler, "adapter":adapter})

    # Iterate boxes
    for i in range(len(det_res.boxes)):
        cls_id = int(det_res.boxes.cls[i].item())
        cname = names[cls_id].lower()

        if cname not in ["person","car","motorcycle","truck","bus","bicycle"]:
            continue

        x1,y1,x2,y2 = det_res.boxes.xyxy[i].cpu().numpy().astype(int)
        x1 = max(0, x1); y1 = max(0, y1)
        x2 = max(0, x2); y2 = max(0, y2)
        box=[x1,y1,x2,y2]

        crop = img_rgb[y1:y2, x1:x2]
        if crop.size==0:
            img_t = val_transform(np.zeros((IMAGE_SIZE,IMAGE_SIZE,3),dtype=np.uint8)).unsqueeze(0).to(DEVICE)
        else:
            img_t = val_transform(crop).unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            emb = backbone(img_t)  # (1, bb_out_dim) or (1, D)
            if emb.dim() == 4:
                emb = torch.nn.functional.adaptive_avg_pool2d(emb, 1).reshape(emb.shape[0], -1)
            emb_np = emb.cpu().numpy()

        # Build full feature vector
        feat_vec = build_44_features(img_rgb, box, cname, wseg_model, pose_model)

        # ensemble prediction
        probs = np.zeros((1,NUM_CLASSES), dtype=np.float32)
        for fm in fold_models:
            Xs = fm["scaler"].transform(feat_vec.reshape(1,-1))
            xp = fm["ngb"].predict_proba(Xs)  # (1, NUM_CLASSES)

            emb_t = torch.from_numpy(emb_np).float().to(DEVICE)  # (1, bb_out_dim)
            if fm.get("adapter", None) is not None:
                with torch.no_grad():
                    emb_t = fm["adapter"](emb_t)

            xp_t = torch.from_numpy(xp).float().to(DEVICE)

            with torch.no_grad():
                logits = fm["fusion"](emb_t, xp_t)
                p = torch.softmax(logits, dim=1).cpu().numpy()

            probs += p

        probs /= max(1, len(fold_models))

        # --- apply meta-classifier filter for class-4 (if available) ---
        try:
            meta_clf_path = Path(OUT_DIR) / "meta_clf_class4.pkl"
            chosen_thr = 0.75  # keep in sync with your validation choice

            if meta_clf_path.exists():
                meta_clf = joblib.load(meta_clf_path)

                # fusion features
                fusion_prob4 = float(probs[0, 4])
                fusion_conf_val = float(probs.max())
                ngb_entropy_val = float(-np.sum(np.clip(probs, 1e-12, 1.0) * np.log(np.clip(probs, 1e-12, 1.0))))

                # proto_sim4: average similarity to prototypes (use first available prototypes file)
                proto_sim4_val = 0.0
                proto_files = []
                for fo in meta:
                    cand = Path(fo["fusion_ckpt"]).parent / f"prototypes_fold{fo['fold']}.npz"
                    if cand.exists():
                        proto_files.append(cand)
                if len(proto_files) > 0:
                    P = np.load(proto_files[0], allow_pickle=True)
                    prototypes = P["prototypes"].item()
                    if 4 in prototypes:
                        prot4 = np.array(prototypes[4], dtype=np.float32)
                        prot4 = prot4 / (np.linalg.norm(prot4) + 1e-12)
                        emb_vec = emb_np.reshape(-1)  # (D,)
                        emb_vec = emb_vec / (np.linalg.norm(emb_vec) + 1e-12)
                        proto_sim4_val = float(np.dot(emb_vec, prot4))

                # assemble meta feature vector (shape (1,4))
                X_meta_box = np.array([[fusion_prob4, fusion_conf_val, proto_sim4_val, ngb_entropy_val]], dtype=np.float32)
                meta_conf = meta_clf.predict_proba(X_meta_box)[:, 1][0]

                # if meta says this predicted-4 should be demoted, zero it out and renormalize
                if int(probs.argmax(axis=1)[0]) == 4 and meta_conf < chosen_thr:
                    probs[0, 4] = 0.0
                    s = probs.sum()
                    if s <= 0:
                        probs[0] = 1.0 / probs.shape[1]
                    else:
                        probs = probs / s
        except Exception:
            # defensive: if anything goes wrong, skip meta-filter and continue
            pass

        pred = int(probs.argmax(axis=1)[0]) if probs.size>0 else 0

        row = {
            "image_path": image_path,
            "class_name": cname,
            "box_x1": x1, "box_y1": y1,
            "box_x2": x2, "box_y2": y2,
            "predicted_level": pred
        }

        # Store full feature vector for debugging/analysis
        for j,f in enumerate(ALL_FEATURES):
            row[f] = float(feat_vec[j])
            # --- Add per-class probabilities and predicted-prob column (NO other changes) ---
            # (uses the `probs` and `pred` variables already computed above)
            for k in range(NUM_CLASSES):
                row[f"prob_L{k}"] = float(probs[0, k])
            row["pred_prob"] = float(probs[0, pred]) if 0 <= pred < NUM_CLASSES else 0.0
            # --- end snippet ---

        rows.append(row)

    df=pd.DataFrame(rows)
    df.to_csv(out_csv, index=False)
    print("Saved inference CSV:", out_csv)
    return df

In [ ]:
df_inf = infer_image(
    "/home/arnab/Desktop/yolo/data/Flood_model/test/1318_jpg.rf.8da76f24cfdcd4d19ccc4fe2f2147ddd.jpg",
    out_csv=f"{OUT_DIR}/inference_1318.csv"
)

In [ ]:
# Cell 9 — Visualize flood-level predictions with bounding boxes (clean version: no LLM, no final-line overlay)
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

def visualize_inference(image_path, df_inf, save_path=None, show_confidence=False, max_display_size=1200):
    """
    Draw boxes from df_inf.
    df_inf must contain: box_x1, box_y1, box_x2, box_y2, class_name, predicted_level,
    and optionally pred_prob or prob_L* columns.
    """

    if df_inf is None or len(df_inf) == 0:
        raise ValueError("df_inf is empty or None — run infer_image() first.")

    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(image_path)

    h0, w0 = img.shape[:2]

    # simple color map for flood levels
    cmap = {}
    for lvl in range(0, 11):
        t = lvl / 10.0
        b = int((1.0 - t) * 0 + t * 0)
        g = int((1.0 - t) * 200 + t * 50)
        r = int((1.0 - t) * 0 + t * 200)
        cmap[lvl] = (b, g, r)

    img_out = img.copy()

    for idx, row in df_inf.iterrows():
        try:
            x1 = int(row["box_x1"])
            y1 = int(row["box_y1"])
            x2 = int(row["box_x2"])
            y2 = int(row["box_y2"])
            cls_name = str(row.get("class_name", "obj"))
            level = int(row.get("predicted_level", 0))
        except Exception:
            continue

        x1 = max(0, min(x1, w0-1))
        y1 = max(0, min(y1, h0-1))
        x2 = max(0, min(x2, w0-1))
        y2 = max(0, min(y2, h0-1))

        color = cmap.get(level, (0, 255, 0))
        cv2.rectangle(img_out, (x1, y1), (x2, y2), color, 2)

        label = f"{cls_name} | L{level}"
        if show_confidence and "pred_prob" in df_inf.columns:
            try:
                prob = float(row.get("pred_prob", 0.0))
                label += f" {prob:.2f}"
            except Exception:
                pass

        text_y = max(12, y1 - 8)
        cv2.putText(
            img_out,
            label,
            (x1, text_y),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            color,
            2,
            cv2.LINE_AA
        )

    # Resize for viewing
    H, W = img_out.shape[:2]
    scale = 1.0
    if max(H, W) > max_display_size:
        scale = max_display_size / max(H, W)
        img_disp = cv2.resize(img_out, (int(W*scale), int(H*scale)))
    else:
        img_disp = img_out

    # Convert BGR to RGB for display
    img_rgb = img_disp[:, :, ::-1]

    plt.figure(figsize=(12, 12 * (img_rgb.shape[0] / img_rgb.shape[1])))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

    if save_path:
        cv2.imwrite(save_path, img_out)
        print("Saved annotated image:", save_path)

    return img_out

In [ ]:
visualize_inference(
    "/home/arnab/Desktop/yolo/data/Flood_model/test/1318_jpg.rf.8da76f24cfdcd4d19ccc4fe2f2147ddd.jpg",
    df_inf,
    save_path=f"{OUT_DIR}/1318_annotated.jpg"
)

In [ ]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

# default: Load the model on the available device(s)
model = Qwen3VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-4B-Instruct", dtype="auto", device_map="auto")
processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-4B-Instruct")

In [ ]:
from PIL import Image
from IPython.display import display

image_path = "/home/arnab/Desktop/yolo/data/Flood_model/yolov11_refined_balanced/object_label_csv2ConvNeXT_NGBoost_FShL+llm/hybrid_resnet50_xgb_fusion_v1/1318_annotated.jpg"
prompt = "Give me the final single L value in one sentence, from this image's bounding boxes (general bounding box value), by seeing the person, car, motorcycle, bicycle, truck, bus in the water."
image = Image.open(image_path)
display(image)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_path,
            },
            {"type": "text", "text": prompt},
        ],
    }
]

# Preparation for inference
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
)

# Move inputs to the same device as the model
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=1000)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs["input_ids"], generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)